<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Performs Research with Multi-Agent Group Chat

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framwork allows tool use and human participance through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [1]:
%%capture --no-stderr
# %pip install pyautogen~=0.1.0

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [1]:
import autogen

## Construct Agents

In [3]:
gpt_config = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0,
    "config_list": [],
    "request_timeout": 360,
}
user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
''',
)
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code."""
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
''',
    llm_config=gpt_config,
)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "autogen_dir"},
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt_config)

In [4]:
groupchat.reset()
manager.reset()

## Start Chat

In [5]:
user_proxy.initiate_chat(
    manager,
    message="""
Using the parquet file named "legal_reddit_df_with_titles.parquet" that is in the current working directory, conduct a clustering analysis using the text found in the "body" column. \
If you receive errors when executing code you are probably going too fast so take a 10 min sleep break.
""",
)

Admin (to chat_manager):


Using the parquet file named "legal_reddit_df_with_titles.parquet" that is in the current working directory, conduct a clustering analysis using the text found in the "body" column. If you receive errors when executing code you are probably going too fast so take a 10 min sleep break.


--------------------------------------------------------------------------------
Planner (to chat_manager):

Plan:

1. Engineer: Load the parquet file "legal_reddit_df_with_titles.parquet" into a DataFrame using a suitable library like pandas or PySpark.

2. Scientist: Inspect the DataFrame to understand its structure and the type of data in the "body" column.

3. Engineer: Preprocess the text data in the "body" column. This may involve removing stop words, punctuation, and applying stemming or lemmatization.

4. Scientist: Decide on the appropriate clustering algorithm to use. This could be K-means, hierarchical clustering, DBSCAN, etc., depending on the nature of the data.



Executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
   created_utc  ...                                          llm_title
0   1588353888  ...  "Legal implications of mass termination and po...
1   1575490057  ...  "Unforeseen snow removal policy changes and po...
2   1589320125  ...  "Tenant seeks legal advice after landlord fail...
3   1589905435  ...  Navigating Wedding Rescheduling and Negotiatin...
4   1578696054  ...  "Legal rights when locked out of home without ...

[5 rows x 10 columns]


--------------------------------------------------------------------------------
[autogen.oai.completion: 10-02 20:00:32] {237} INFO - retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 207, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\opena

INFO:autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 207, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 700, in _interpret_respo

[autogen.oai.completion: 10-02 20:00:42] {237} INFO - retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 207, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", lin

INFO:autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 207, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 700, in _interpret_respo

...

INFO:autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 207, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 700, in _interpret_respo

[autogen.oai.completion: 10-02 20:02:00] {237} INFO - retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 207, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", lin

INFO:autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 207, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 700, in _interpret_respo

[autogen.oai.completion: 10-02 20:02:10] {237} INFO - retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 207, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", lin

INFO:autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 207, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 700, in _interpret_respo

In [ ]:
%pip install "pyautogen[retrievechat]~=0.1.2" "flaml[automl]"

In [ ]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

autogen.ChatCompletion.start_logging()

# 1. create an RetrieveAssistantAgent instance named "assistant"
assistant = RetrieveAssistantAgent(
    name="assistant", 
    system_message="You are a helpful assistant.",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
)

# 2. create the RetrieveUserProxyAgent instance named "ragproxyagent"
# By default, the human_input_mode is "ALWAYS", which means the agent will ask for human input at every step. We set it to "NEVER" here.
# `docs_path` is the path to the docs directory. By default, it is set to "./docs". Here we generated the documentations from FLAML's docstrings.
# Navigate to the website folder and run `pydoc-markdown` and it will generate folder `reference` under `website/docs`.
# `task` indicates the kind of task we're working on. In this example, it's a `code` task.
# `chunk_token_size` is the chunk token size for the retrieve chat. By default, it is set to `max_tokens * 0.6`, here we set it to 2000.
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "task": "code",
        "docs_path": "../website/docs/reference",
        "chunk_token_size": 2000,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "embedding_model": "all-mpnet-base-v2",
    },
)

## Create Group Chat without Critic for Comparison

In [6]:
groupchat_nocritic = autogen.GroupChat(agents=[user_proxy, engineer, scientist, planner, executor], messages=[], max_round=25)
for agent in groupchat.agents:
    agent.reset()
manager_nocritic = autogen.GroupChatManager(groupchat=groupchat_nocritic, llm_config=gpt4_config)
user_proxy.initiate_chat(
    manager_nocritic,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------
Planner (to chat_manager):

Plan:

1. Engineer: Write a script to scrape the arXiv website for papers related to LLM (Language Model) applications published in the last week. The script should extract the title, authors, abstract, and link to the paper.

2. Scientist: Review the scraped data to ensure it's relevant to LLM applications.

3. Engineer: Write a script to convert the scraped data into a markdown table. The table should include columns for the title, authors, abstract, and link to the paper.

4. Scientist: Review the markdown table to ensure it's correctly formatted and the information is accurate.

5. Engineer: Make any necessary revisions to the markdown table based on the scientist's feedback.

6. Scientist: Once the markdown table is finalized, categorize ea

Executor (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
  File "", line 1
    !pip install arxiv pandas tabulate
    ^
SyntaxError: invalid syntax


--------------------------------------------------------------------------------
Engineer (to chat_manager):

I apologize for the confusion. The `!pip install` command is specific to Jupyter notebooks and cannot be used in this environment. 

Let's modify the code to install the packages using Python's `subprocess` module, which allows us to spawn new processes, connect to their input/output/error pipes, and obtain their return codes.

```python
import subprocess

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install('arxiv')
install('pandas')
install('tabulate')

import arxiv
import pandas as pd
from tabulate import tabulate
from datetime import datetime, timedelta

# Define the date a week ago
week_ago = datetime.now() - timedelta(weeks=1)

# Search for papers relate

Executor (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 6, in <module>
    install('arxiv')
  File "", line 4, in install
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
NameError: name 'sys' is not defined


--------------------------------------------------------------------------------
Engineer (to chat_manager):

I apologize for the oversight. The `sys` module was not imported. Let's correct this by importing the `sys` module.

```python
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install('arxiv')
install('pandas')
install('tabulate')

import arxiv
import pandas as pd
from tabulate import tabulate
from datetime import datetime, timedelta

# Define the date a week ago
week_ago = datetime.now() - timedelta(weeks=1)

# Search for papers related to LLM applications published in the last week
search = arxi

Executor (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
:35: DeprecationWarning: The 'get' method is deprecated, use 'results' instead
  for result in search.get():
Traceback (most recent call last):
  File "", line 37, in <module>
    if result.published >= week_ago:
TypeError: can't compare offset-naive and offset-aware datetimes


--------------------------------------------------------------------------------
[autogen.oai.completion: 09-29 19:37:57] {236} INFO - retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 206, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-p

INFO:autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 206, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 700, in _interpret_respo

...

[autogen.oai.completion: 09-29 19:59:35] {236} INFO - retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 206, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", lin

INFO:autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 206, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 700, in _interpret_respo

[autogen.oai.completion: 09-29 19:59:45] {236} INFO - retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 206, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", lin

INFO:autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\autogen\oai\completion.py", line 206, in _get_response
    response = openai_completion.create(**config)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_resources\abstract\engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\openai\api_requestor.py", line 700, in _interpret_respo

RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-MDdREazIjXwusGUQAVXvfijt on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.

In [8]:
from IPython.display import Markdown, display, HTML

def read_file(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        data = file.read()
    return data

# Use the function to read the file
data = read_file("paper/LLM_applications.txt")
Markdown(f"{data}")

| Title                                                                                                                                        | Authors                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | Abstract                                                                        | Link                              | Date                      |
|:---------------------------------------------------------------------------------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:--------------------------------------------------------------------------------|:----------------------------------|:--------------------------|
| Demystifying CLIP Data                                                                                                                       | Hu Xu, Saining Xie, Xiaoqing Ellen Tan, Po-Yao Huang, Russell Howes, Vasu Sharma, Shang-Wen Li, Gargi Ghosh, Luke Zettlemoyer, Christoph Feichtenhofer                                                                                                                                                                                                                                                                                                                                                                                                                                      | Contrastive Language-Image Pre-training (CLIP) is an approach that has          | http://arxiv.org/abs/2309.16671v1 | 2023-09-28 17:59:56+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | advanced research and applications in computer vision, fueling modern           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | recognition systems and generative models. We believe that the main ingredient  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to the success of CLIP is its data and not the model architecture or            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pre-training objective. However, CLIP only provides very limited information    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | about its data and how it has been collected, leading to works that aim to      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | reproduce CLIP's data by filtering with its model parameters. In this work, we  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | intend to reveal CLIP's data curation approach and in our pursuit of making it  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | open to the community introduce Metadata-Curated Language-Image Pre-training    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (MetaCLIP). MetaCLIP takes a raw data pool and metadata (derived from CLIP's    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | concepts) and yields a balanced subset over the metadata distribution. Our      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | experimental study rigorously isolates the model and training settings,         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | concentrating solely on data. MetaCLIP applied to CommonCrawl with 400M         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | image-text data pairs outperforms CLIP's data on multiple standard benchmarks.  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | In zero-shot ImageNet classification, MetaCLIP achieves 70.8% accuracy,         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | surpassing CLIP's 68.3% on ViT-B models. Scaling to 1B data, while maintaining  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the same training budget, attains 72.4%. Our observations hold across various   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | model sizes, exemplified by ViT-H achieving 80.5%, without any                  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | bells-and-whistles. Curation code and training data distribution on metadata is |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | made available at https://github.com/facebookresearch/MetaCLIP.                 |                                   |                           |
| Decaf: Monocular Deformation Capture for Face and Hand Interactions                                                                          | Soshi Shimada, Vladislav Golyanik, Patrick Pérez, Christian Theobalt                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | Existing methods for 3D tracking from monocular RGB videos predominantly        | http://arxiv.org/abs/2309.16670v1 | 2023-09-28 17:59:51+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | consider articulated and rigid objects. Modelling dense non-rigid object        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | deformations in this setting remained largely unaddressed so far, although such |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | effects can improve the realism of the downstream applications such as AR/VR    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and avatar communications. This is due to the severe ill-posedness of the       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | monocular view setting and the associated challenges. While it is possible to   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | naively track multiple non-rigid objects independently using 3D templates or    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | parametric 3D models, such an approach would suffer from multiple artefacts in  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the resulting 3D estimates such as depth ambiguity, unnatural intra-object      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | collisions and missing or implausible deformations. Hence, this paper           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | introduces the first method that addresses the fundamental challenges depicted  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | above and that allows tracking human hands interacting with human faces in 3D   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | from single monocular RGB videos. We model hands as articulated objects         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | inducing non-rigid face deformations during an active interaction. Our method   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | relies on a new hand-face motion and interaction capture dataset with realistic |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | face deformations acquired with a markerless multi-view camera system. As a     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pivotal step in its creation, we process the reconstructed raw 3D shapes with   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | position-based dynamics and an approach for non-uniform stiffness estimation of |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the head tissues, which results in plausible annotations of the surface         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | deformations, hand-face contact regions and head-hand positions. At the core of |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | our neural approach are a variational auto-encoder supplying the hand-face      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | depth prior and modules that guide the 3D tracking by estimating the contacts   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and the deformations. Our final 3D hand and face reconstructions are realistic  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and more plausible compared to several baselines applicable in our setting,     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | both quantitatively and qualitatively.                                          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | https://vcai.mpi-inf.mpg.de/projects/Decaf                                      |                                   |                           |
| Asymptotically Optimal Sequential Multiple Testing Procedures for Correlated Normal                                                          | Monitirtha Dey, Subir Kumar Bhandari                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | Simultaneous statistical inference has been a cornerstone in the statistics     | http://arxiv.org/abs/2309.16657v1 | 2023-09-28 17:55:49+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | methodology literature because of its fundamental theory and paramount          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications. The mainstream multiple testing literature has traditionally      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | considered two frameworks: the sample size is deterministic, and the test       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | statistics corresponding to different tests are independent. However, in many   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | modern scientific avenues, these assumptions are often violated. There is       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | little study that explores the multiple testing problem in a sequential         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | framework where the test statistics corresponding to the various streams are    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | dependent. This work fills this gap in a unified way by considering the         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | classical means-testing problem in an equicorrelated Gaussian and sequential    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | framework. We focus on sequential test procedures that control the type I and   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | type II familywise error probabilities at pre-specified levels. We establish    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | that our proposed test procedures achieve the optimal expected sample sizes     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | under every possible signal configuration asymptotically, as the two error      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | probabilities vanish at arbitrary rates. Towards this, we elucidate that the    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ratio of the expected sample size of our proposed rule and that of the          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | classical SPRT goes to one asymptotically, thus illustrating their connection.  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Generalizing this, we show that our proposed procedures, with appropriately     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | adjusted critical values, are asymptotically optimal for controlling any        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | multiple testing error metric lying between multiples of FWER in a certain      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | sense. This class of metrics includes FDR/FNR, pFDR/pFNR, the per-comparison    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and per-family error rates, and the false positive rate.                        |                                   |                           |
| Visual In-Context Learning for Few-Shot Eczema Segmentation                                                                                  | Neelesh Kumar, Oya Aran, Venugopal Vasudevan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | Automated diagnosis of eczema from digital camera images is crucial for         | http://arxiv.org/abs/2309.16656v1 | 2023-09-28 17:55:24+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | developing applications that allow patients to self-monitor their recovery. An  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | important component of this is the segmentation of eczema region from such      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | images. Current methods for eczema segmentation rely on deep neural networks    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | such as convolutional (CNN)-based U-Net or transformer-based Swin U-Net. While  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | effective, these methods require high volume of annotated data, which can be    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | difficult to obtain. Here, we investigate the capabilities of visual in-context |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | learning that can perform few-shot eczema segmentation with just a handful of   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | examples and without any need for retraining models. Specifically, we propose a |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | strategy for applying in-context learning for eczema segmentation with a        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | generalist vision model called SegGPT. When benchmarked on a dataset of         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | annotated eczema images, we show that SegGPT with just 2 representative example |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | images from the training dataset performs better (mIoU: 36.69) than a CNN U-Net |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | trained on 428 images (mIoU: 32.60). We also discover that using more number of |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | examples for SegGPT may in fact be harmful to its performance. Our result       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | highlights the importance of visual in-context learning in developing faster    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and better solutions to skin imaging tasks. Our result also paves the way for   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | developing inclusive solutions that can cater to minorities in the demographics |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | who are typically heavily under-represented in the training data.               |                                   |                           |
| DreamGaussian: Generative Gaussian Splatting for Efficient 3D Content Creation                                                               | Jiaxiang Tang, Jiawei Ren, Hang Zhou, Ziwei Liu, Gang Zeng                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Recent advances in 3D content creation mostly leverage optimization-based 3D    | http://arxiv.org/abs/2309.16653v1 | 2023-09-28 17:55:05+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | generation via score distillation sampling (SDS). Though promising results have |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | been exhibited, these methods often suffer from slow per-sample optimization,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | limiting their practical usage. In this paper, we propose DreamGaussian, a      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | novel 3D content generation framework that achieves both efficiency and quality |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | simultaneously. Our key insight is to design a generative 3D Gaussian Splatting |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | model with companioned mesh extraction and texture refinement in UV space. In   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | contrast to the occupancy pruning used in Neural Radiance Fields, we            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | demonstrate that the progressive densification of 3D Gaussians converges        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | significantly faster for 3D generative tasks. To further enhance the texture    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quality and facilitate downstream applications, we introduce an efficient       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | algorithm to convert 3D Gaussians into textured meshes and apply a fine-tuning  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | stage to refine the details. Extensive experiments demonstrate the superior     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | efficiency and competitive generation quality of our proposed approach.         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Notably, DreamGaussian produces high-quality textured meshes in just 2 minutes  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | from a single-view image, achieving approximately 10 times acceleration         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | compared to existing methods.                                                   |                                   |                           |
| FLIP: Cross-domain Face Anti-spoofing with Language Guidance                                                                                 | Koushik Srivatsan, Muzammal Naseer, Karthik Nandakumar                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Face anti-spoofing (FAS) or presentation attack detection is an essential       | http://arxiv.org/abs/2309.16649v1 | 2023-09-28 17:53:20+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | component of face recognition systems deployed in security-critical             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications. Existing FAS methods have poor generalizability to unseen spoof   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | types, camera sensors, and environmental conditions. Recently, vision           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | transformer (ViT) models have been shown to be effective for the FAS task due   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to their ability to capture long-range dependencies among image patches.        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | However, adaptive modules or auxiliary loss functions are often required to     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | adapt pre-trained ViT weights learned on large-scale datasets such as ImageNet. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | In this work, we first show that initializing ViTs with multimodal (e.g., CLIP) |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pre-trained weights improves generalizability for the FAS task, which is in     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | line with the zero-shot transfer capabilities of vision-language pre-trained    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (VLP) models. We then propose a novel approach for robust cross-domain FAS by   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | grounding visual representations with the help of natural language.             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Specifically, we show that aligning the image representation with an ensemble   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of class descriptions (based on natural language semantics) improves FAS        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | generalizability in low-data regimes. Finally, we propose a multimodal          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | contrastive learning strategy to boost feature generalization further and       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | bridge the gap between source and target domains. Extensive experiments on      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | three standard protocols demonstrate that our method significantly outperforms  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the state-of-the-art methods, achieving better zero-shot transfer performance   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | than five-shot transfer of adaptive ViTs. Code:                                 |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | https://github.com/koushiksrivats/FLIP                                          |                                   |                           |
| MindShift: Leveraging Large Language Models for Mental-States-Based Problematic Smartphone Use Intervention                                  | Ruolan Wu, Chun Yu, Xiaole Pan, Yujia Liu, Ningning Zhang, Yue Fu, Yuhan Wang, Zhi Zheng, Li Chen, Qiaolei Jiang, Xuhai Xu, Yuanchun Shi                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Problematic smartphone use negatively affects physical and mental health.       | http://arxiv.org/abs/2309.16639v1 | 2023-09-28 17:49:03+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Despite the wide range of prior research, existing persuasive techniques are    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | not flexible enough to provide dynamic persuasion content based on users'       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | physical contexts and mental states. We first conduct a Wizard-of-Oz study      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (N=12) and an interview study (N=10) to summarize the mental states behind      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problematic smartphone use: boredom, stress, and inertia. This informs our      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | design of four persuasion strategies: understanding, comforting, evoking, and   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | scaffolding habits. We leverage large language models (LLMs) to enable the      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | automatic and dynamic generation of effective persuasion content. We develop    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | MindShift, a novel LLM-powered problematic smartphone use intervention          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | technique. MindShift takes users' in-the-moment physical contexts, mental       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | states, app usage behaviors, users' goals & habits as input, and generates      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | high-quality and flexible persuasive content with appropriate persuasion        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | strategies. We conduct a 5-week field experiment (N=25) to compare MindShift    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | with baseline techniques. The results show that MindShift significantly         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | improves intervention acceptance rates by 17.8-22.5% and reduces smartphone use |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | frequency by 12.1-14.4%. Moreover, users have a significant drop in smartphone  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | addiction scale scores and a rise in self-efficacy. Our study sheds light on    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the potential of leveraging LLMs for context-aware persuasion in other behavior |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | change domains.                                                                 |                                   |                           |
| Analysis of the Usability of Automatically Enriched Cultural Heritage Data                                                                   | Julien Antoine Raemy, Robert Sanderson                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | This chapter presents the potential of interoperability and standardised data   | http://arxiv.org/abs/2309.16635v1 | 2023-09-28 17:43:00+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | publication for cultural heritage resources, with a focus on community-driven   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | approaches and web standards for usability. The Linked Open Usable Data (LOUD)  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | design principles, which rely on JSON-LD as lingua franca, serve as the         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | foundation.                                                                     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |   We begin by exploring the significant advances made by the International      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Image Interoperability Framework (IIIF) in promoting interoperability for       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | image-based resources. The principles and practices of IIIF have paved the way  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | for Linked Art, which expands the use of linked data by demonstrating how it    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | can easily facilitate the integration and sharing of semantic cultural heritage |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | data across portals and institutions.                                           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |   To provide a practical demonstration of the concepts discussed, the chapter   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | highlights the implementation of LUX, the Yale Collections Discovery platform.  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | LUX serves as a compelling case study for the use of linked data at scale,      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | demonstrating the real-world application of automated enrichment in the         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | cultural heritage domain.                                                       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |   Rooted in empirical study, the analysis presented in this chapter delves into |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the broader context of community practices and semantic interoperability. By    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | examining the collaborative efforts and integration of diverse cultural         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | heritage resources, the research sheds light on the potential benefits and      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | challenges associated with LOUD.                                                |                                   |                           |
| On the Role of 5G and Beyond Sidelink Communication in Multi-Hop Tactical Networks                                                           | Charles E. Thornton, Evan Allen, Evar Jones, Daniel Jakubisin, Fred Templin, Lingjia Liu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | This work investigates the potential of 5G and beyond sidelink (SL)             | http://arxiv.org/abs/2309.16628v1 | 2023-09-28 17:32:20+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | communication to support multi-hop tactical networks. We first provide a        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | technical and historical overview of 3GPP SL standardization activities, and    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | then consider applications to current problems of interest in tactical          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | networking. We consider a number of multi-hop routing techniques which are      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | expected to be of interest for SL-enabled multi-hop tactical networking and     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | examine open-source tools useful for network emulation. Finally, we discuss     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | relevant research directions which may be of interest for 5G SL-enabled         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | tactical communications, namely the integration of RF sensing and positioning,  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | as well as emerging machine learning tools such as federated and decentralized  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | learning, which may be of great interest for resource allocation and routing    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problems that arise in tactical applications. We conclude by summarizing recent |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | developments in the 5G SL literature and provide guidelines for future          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | research.                                                                       |                                   |                           |
| On generalised majority edge-colourings of graphs                                                                                            | Paweł Pękała, Jakub Przybyło                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | A $\frac{1}{k}$-majority $l$-edge-colouring of a graph $G$ is a colouring of    | http://arxiv.org/abs/2309.16624v1 | 2023-09-28 17:24:17+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | its edges with $l$ colours such that for every colour $i$ and each vertex $v$   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of $G$, at most $\frac{1}{k}$'th of the edges incident with $v$ have colour     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | $i$. We conjecture that for every integer $k\geq 2$, each graph with minimum    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | degree $\delta\geq k^2$ is $\frac{1}{k}$-majority $(k+1)$-edge-colourable and   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | observe that such result would be best possible. This was already known to hold |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | for $k=2$. We support the conjecture by proving it with $2k^2$ instead of       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | $k^2$, which confirms the right order of magnitude of the conjectured optimal   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | lower bound for $\delta$. We at the same time improve the previously known      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | bound of order $k^3\log k$, based on a straightforward probabilistic approach.  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | As this technique seems not applicable towards any further improvement, we use  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | a more direct non-random approach. We also strengthen our result, in particular |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | substituting $2k^2$ by $(\frac{7}{4}+o(1))k^2$. Finally, we provide the proof   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of the conjecture itself for $k\leq 4$ and completely solve an analogous        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problem for the family of bipartite graphs.                                     |                                   |                           |
| Stress Testing Chain-of-Thought Prompting for Large Language Models                                                                          | Aayush Mishra, Karan Thakkar                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | This report examines the effectiveness of Chain-of-Thought (CoT) prompting in   | http://arxiv.org/abs/2309.16621v1 | 2023-09-28 17:21:33+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | improving the multi-step reasoning abilities of large language models (LLMs).   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Inspired by previous studies \cite{Min2022RethinkingWork}, we analyze the       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | impact of three types of CoT prompt perturbations, namely CoT order, CoT        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | values, and CoT operators on the performance of GPT-3 on various tasks. Our     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | findings show that incorrect CoT prompting leads to poor performance on         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | accuracy metrics. Correct values in the CoT is crucial for predicting correct   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | answers. Moreover, incorrect demonstrations, where the CoT operators or the CoT |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | order are wrong, do not affect the performance as drastically when compared to  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the value based perturbations. This research deepens our understanding of CoT   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | prompting and opens some new questions regarding the capability of LLMs to      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | learn reasoning in context.                                                     |                                   |                           |
| Cubical Approximation for Directed Topology II                                                                                               | Sanjeevi Krishnan                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | The paper establishes an equivalence between localizations of (diagrams of)     | http://arxiv.org/abs/2309.16619v1 | 2023-09-28 17:19:04+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | cubical sets and (diagrams of) directed topological spaces by those maps        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | defining (natural) cubical homotopy equivalences after application of the       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | directed singular functor and a directed analogue of fibrant replacement. This  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | equivalence both lifts and extends an equivalence between classical homotopy    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | categories of cubical sets and topological spaces. Some simple applications     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | include combinatorial descriptions and subsequent calculations of directed      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | homotopy monoids and directed singular 1-cohomology monoids. Another            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | application is a characterization of isomorphisms between small categories up   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to zig-zags of natural transformations as directed homotopy equivalences        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | between directed classifying spaces. Cubical sets throughout the paper are      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | taken to mean presheaves over the minimal symmetric monoidal variant of the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | cube category. Along the way, the paper characterizes morphisms in this variant |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | as the interval-preserving lattice homomorphisms between finite Boolean lattice |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and describes some of the test model structure on presheaves over this variant. |                                   |                           |
| Revisiting Neural Program Smoothing for Fuzzing                                                                                              | Maria-Irina Nicolae, Max Eisele, Andreas Zeller                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Testing with randomly generated inputs (fuzzing) has gained significant         | http://arxiv.org/abs/2309.16618v1 | 2023-09-28 17:17:11+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | traction due to its capacity to expose program vulnerabilities automatically.   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Fuzz testing campaigns generate large amounts of data, making them ideal for    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the application of machine learning (ML). Neural program smoothing (NPS), a     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | specific family of ML-guided fuzzers, aims to use a neural network as a smooth  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | approximation of the program target for new test case generation.               |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |   In this paper, we conduct the most extensive evaluation of NPS fuzzers        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | against standard gray-box fuzzers (>11 CPU years and >5.5 GPU years), and make  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the following contributions: (1) We find that the original performance claims   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | for NPS fuzzers do not hold; a gap we relate to fundamental, implementation,    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and experimental limitations of prior works. (2) We contribute the first        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in-depth analysis of the contribution of machine learning and gradient-based    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | mutations in NPS. (3) We implement Neuzz++, which shows that addressing the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | practical limitations of NPS fuzzers improves performance, but that standard    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | gray-box fuzzers almost always surpass NPS-based fuzzers. (4) As a consequence, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | we propose new guidelines targeted at benchmarking fuzzing based on machine     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | learning, and present MLFuzz, a platform with GPU access for easy and           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | reproducible evaluation of ML-based fuzzers. Neuzz++, MLFuzz, and all our data  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | are public.                                                                     |                                   |                           |
| Qwen Technical Report                                                                                                                        | Jinze Bai, Shuai Bai, Yunfei Chu, Zeyu Cui, Kai Dang, Xiaodong Deng, Yang Fan, Wenbin Ge, Yu Han, Fei Huang, Binyuan Hui, Luo Ji, Mei Li, Junyang Lin, Runji Lin, Dayiheng Liu, Gao Liu, Chengqiang Lu, Keming Lu, Jianxin Ma, Rui Men, Xingzhang Ren, Xuancheng Ren, Chuanqi Tan, Sinan Tan, Jianhong Tu, Peng Wang, Shijie Wang, Wei Wang, Shengguang Wu, Benfeng Xu, Jin Xu, An Yang, Hao Yang, Jian Yang, Shusheng Yang, Yang Yao, Bowen Yu, Hongyi Yuan, Zheng Yuan, Jianwei Zhang, Xingxuan Zhang, Yichang Zhang, Zhenru Zhang, Chang Zhou, Jingren Zhou, Xiaohuan Zhou, Tianhang Zhu | Large language models (LLMs) have revolutionized the field of artificial        | http://arxiv.org/abs/2309.16609v1 | 2023-09-28 17:07:49+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | intelligence, enabling natural language processing tasks that were previously   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | thought to be exclusive to humans. In this work, we introduce Qwen, the first   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | installment of our large language model series. Qwen is a comprehensive         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | language model series that encompasses distinct models with varying parameter   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | counts. It includes Qwen, the base pretrained language models, and Qwen-Chat,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the chat models finetuned with human alignment techniques. The base language    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | models consistently demonstrate superior performance across a multitude of      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | downstream tasks, and the chat models, particularly those trained using         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Reinforcement Learning from Human Feedback (RLHF), are highly competitive. The  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | chat models possess advanced tool-use and planning capabilities for creating    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | agent applications, showcasing impressive performance even when compared to     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | bigger models on complex tasks like utilizing a code interpreter. Furthermore,  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | we have developed coding-specialized models, Code-Qwen and Code-Qwen-Chat, as   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | well as mathematics-focused models, Math-Qwen-Chat, which are built upon base   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | language models. These models demonstrate significantly improved performance in |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | comparison with open-source models, and slightly fall behind the proprietary    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | models.                                                                         |                                   |                           |
| Exploiting Edge Features in Graphs with Fused Network Gromov-Wasserstein Distance                                                            | Junjie Yang, Matthieu Labeau, Florence d'Alché-Buc                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Pairwise comparison of graphs is key to many applications in Machine learning   | http://arxiv.org/abs/2309.16604v1 | 2023-09-28 17:05:03+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ranging from clustering, kernel-based classification/regression and more        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | recently supervised graph prediction. Distances between graphs usually rely on  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | informative representations of these structured objects such as bag of          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | substructures or other graph embeddings. A recently popular solution consists   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in representing graphs as metric measure spaces, allowing to successfully       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | leverage Optimal Transport, which provides meaningful distances allowing to     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | compare them: the Gromov-Wasserstein distances. However, this family of         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | distances overlooks edge attributes, which are essential for many structured    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | objects. In this work, we introduce an extension of Gromov-Wasserstein distance |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | for comparing graphs whose both nodes and edges have features. We propose novel |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | algorithms for distance and barycenter computation. We empirically show the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | effectiveness of the novel distance in learning tasks where graphs occur in     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | either input space or output space, such as classification and graph            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | prediction.                                                                     |                                   |                           |
| Constraining Ultralight Dark Matter through an Accelerated Resonant Search                                                                   | Zitong Xu, Xiaolin Ma, Kai Wei, Yuxuan He, Xing Heng, Xiaofei Huang, Tengyu Ai, Jian Liao, Wei Ji, Jia Liu, Xiao-Ping Wang, Dmitry Budker                                                                                                                                                                                                                                                                                                                                                                                                                                                   | Experiments aimed at detecting ultralight dark matter typically rely on         | http://arxiv.org/abs/2309.16600v1 | 2023-09-28 17:03:26+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | resonant effects, which are sensitive to the dark matter mass that matches the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | resonance frequency. In this study, we investigate the nucleon couplings of     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ultralight axion dark matter using a magnetometer operating in a nuclear        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | magnetic resonance (NMR) mode. Our approach involves the use of a $^{21}$Ne     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | spin-based sensor, which features the lowest nuclear magnetic moment among      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | noble-gas spins. This configuration allows us to achieve an ultrahigh           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | sensitivity of 0.73 fT/Hz$^{1/2}$ at around 5 Hz, corresponding to energy       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | resolution of approximately 1.5$\times                                          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |   10^{-23}\,\rm{eV/Hz^{1/2}}$. Our analysis reveals that under certain          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | conditions it is beneficial to scan the frequency with steps significantly      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | larger than the resonance width. The analytical results are in agreement with   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | experimental data and the scan strategy is potentially applicable to other      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | resonant searches. Further, our study establishes stringent constraints on      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | axion-like particles (ALP) in the 4.5--15.5 Hz Compton-frequency range coupling |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to neutrons and protons, improving on prior work by several-fold. Within a band |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | around 4.6--6.6 Hz and around 7.5 Hz, our laboratory findings surpass           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | astrophysical limits derived from neutron-star cooling. Hence, we demonstrate   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | an accelerated resonance search for ultralight dark matter, achieving an        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | approximately 30-fold increase in scanning step while maintaining competitive   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | sensitivity.                                                                    |                                   |                           |
| Can LLMs Effectively Leverage Structural Information for Graph Learning: When and Why                                                        | Jin Huang, Xingjian Zhang, Qiaozhu Mei, Jiaqi Ma                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | This paper studies Large Language Models (LLMs) for structured                  | http://arxiv.org/abs/2309.16595v1 | 2023-09-28 16:58:37+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | data--particularly graphs--a crucial data modality that remains underexplored   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in the LLM literature. We aim to understand when and why the incorporation of   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | structural information inherent in graph data can improve the prediction        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance of LLMs on node classification tasks. To address the ``when''       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | question, we examine a variety of prompting methods for encoding structural     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | information, in settings where textual node features are either rich or scarce. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | For the ``why'' questions, we probe into two potential contributing factors to  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the LLM performance: data leakage and homophily. Our exploration of these       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | questions reveals that (i) LLMs can benefit from structural information,        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | especially when textual node features are scarce; (ii) there is no substantial  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | evidence indicating that the performance of LLMs is significantly attributed to |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | data leakage; and (iii) the performance of LLMs on a target node is strongly    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | positively related to the local homophily ratio of the node.                    |                                   |                           |
| Navigating Healthcare Insights: A Birds Eye View of Explainability with Knowledge Graphs                                                     | Satvik Garg, Shivam Parikh, Somya Garg                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Knowledge graphs (KGs) are gaining prominence in Healthcare AI, especially in   | http://arxiv.org/abs/2309.16593v1 | 2023-09-28 16:57:03+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | drug discovery and pharmaceutical research as they provide a structured way to  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | integrate diverse information sources, enhancing AI system interpretability.    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | This interpretability is crucial in healthcare, where trust and transparency    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | matter, and eXplainable AI (XAI) supports decision making for healthcare        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | professionals. This overview summarizes recent literature on the impact of KGs  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in healthcare and their role in developing explainable AI models. We cover KG   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | workflow, including construction, relationship extraction, reasoning, and their |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications in areas like Drug-Drug Interactions (DDI), Drug Target            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Interactions (DTI), Drug Development (DD), Adverse Drug Reactions (ADR), and    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | bioinformatics. We emphasize the importance of making KGs more interpretable    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | through knowledge-infused learning in healthcare. Finally, we highlight         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | research challenges and provide insights for future directions.                 |                                   |                           |
| GPT-Fathom: Benchmarking Large Language Models to Decipher the Evolutionary Path towards GPT-4 and Beyond                                    | Shen Zheng, Yuyu Zhang, Yijie Zhu, Chenguang Xi, Pengyang Gao, Xun Zhou, Kevin Chen-Chuan Chang                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | With the rapid advancement of large language models (LLMs), there is a          | http://arxiv.org/abs/2309.16583v1 | 2023-09-28 16:43:35+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pressing need for a comprehensive evaluation suite to assess their capabilities |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and limitations. Existing LLM leaderboards often reference scores reported in   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | other papers without consistent settings and prompts, which may inadvertently   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | encourage cherry-picking favored settings and prompts for better results. In    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | this work, we introduce GPT-Fathom, an open-source and reproducible LLM         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | evaluation suite built on top of OpenAI Evals. We systematically evaluate 10+   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | leading LLMs as well as OpenAI's legacy models on 20+ curated benchmarks across |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 7 capability categories, all under aligned settings. Our retrospective study on |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | OpenAI's earlier models offers valuable insights into the evolutionary path     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | from GPT-3 to GPT-4. Currently, the community is eager to know how GPT-3        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | progressively improves to GPT-4, including technical details like whether       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | adding code data improves LLM's reasoning capability, which aspects of LLM      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | capability can be improved by SFT and RLHF, how much is the alignment tax, etc. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Our analysis sheds light on many of these questions, aiming to improve the      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | transparency of advanced LLMs.                                                  |                                   |                           |
| A Physics Informed Machine Learning Method for Power System Model Parameter Optimization                                                     | Georg Kordowich, Johann Jaeger                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | This paper proposes a gradient descent based optimization method that relies    | http://arxiv.org/abs/2309.16579v1 | 2023-09-28 16:34:34+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | on automatic differentiation for the computation of gradients. The method uses  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | tools and techniques originally developed in the field of artificial neural     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | networks and applies them to power system simulations. It can be used as a      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | one-shot physics informed machine learning approach for the identification of   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | uncertain power system simulation parameters. Additionally, it can optimize     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | parameters with respect to a desired system behavior. The paper focuses on      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | presenting the theoretical background and showing exemplary use-cases for both  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | parameter identification and optimization using a single machine infinite       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | busbar system. The results imply a generic applicability for a wide range of    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problems.                                                                       |                                   |                           |
| A Benchmark for Learning to Translate a New Language from One Grammar Book                                                                   | Garrett Tanzer, Mirac Suzgun, Eline Visser, Dan Jurafsky, Luke Melas-Kyriazi                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | Large language models (LLMs) can perform impressive feats with in-context       | http://arxiv.org/abs/2309.16575v1 | 2023-09-28 16:32:28+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | learning or lightweight finetuning. It is natural to wonder how well these      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | models adapt to genuinely new tasks, but how does one find tasks that are       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | unseen in internet-scale training sets? We turn to a field that is explicitly   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | motivated and bottlenecked by a scarcity of web data: low-resource languages.   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | In this paper, we introduce MTOB (Machine Translation from One Book), a         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | benchmark for learning to translate between English and Kalamang -- a language  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | with less than 200 speakers and therefore virtually no presence on the web --   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | using several hundred pages of field linguistics reference materials. This task |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | framing is novel in that it asks a model to learn a language from a single      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | human-readable book of grammar explanations, rather than a large mined corpus   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of in-domain data, more akin to L2 learning than L1 acquisition. We demonstrate |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | that baselines using current LLMs are promising but fall short of human         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance, achieving 44.7 chrF on Kalamang to English translation and 45.8    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | chrF on English to Kalamang translation, compared to 51.6 and 57.0 chrF by a    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | human who learned Kalamang from the same reference materials. We hope that MTOB |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | will help measure LLM capabilities along a new dimension, and that the methods  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | developed to solve it could help expand access to language technology for       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | underserved communities by leveraging qualitatively different kinds of data     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | than traditional machine translation.                                           |                                   |                           |
| Good Moduli Spaces in Derived Algebraic Geometry                                                                                             | Eric Ahlqvist, Jeroen Hekking, Michele Pernice, Michail Savvas                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | We develop a theory of good moduli spaces for derived Artin stacks, which       | http://arxiv.org/abs/2309.16574v1 | 2023-09-28 16:31:45+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | naturally generalizes the classical theory of good moduli spaces introduced by  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Alper. As such, many of the fundamental results and properties regarding good   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | moduli spaces for classical Artin stacks carry over to the derived context. In  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fact, under natural assumptions, often satisfied in practice, we show that the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | derived theory essentially reduces to the classical theory. As applications, we |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | establish derived versions of the \'{e}tale slice theorem for good moduli       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | spaces and the partial desingularization procedure of good moduli spaces.       |                                   |                           |
| Review of Machine Learning Methods for Additive Manufacturing of Functionally Graded Materials                                               | Mohammad Karimzadeh, Aleksandar Vakanski, Fei Xu, Xinchang Zhang                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | Additive manufacturing has revolutionized the manufacturing of complex parts    | http://arxiv.org/abs/2309.16571v1 | 2023-09-28 16:27:07+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | by enabling direct material joining and offers several advantages such as       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | cost-effective manufacturing of complex parts, reducing manufacturing waste,    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and opening new possibilities for manufacturing automation. One group of        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | materials for which additive manufacturing holds great potential for enhancing  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | component performance and properties is Functionally Graded Materials (FGMs).   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | FGMs are advanced composite materials that exhibit smoothly varying properties  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | making them desirable for applications in aerospace, automobile, biomedical,    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and defense industries. Such composition differs from traditional composite     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | materials, since the location-dependent composition changes gradually in FGMs,  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | leading to enhanced properties. Recently, machine learning techniques have      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | emerged as a promising means for fabrication of FGMs through optimizing         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | processing parameters, improving product quality, and detecting manufacturing   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | defects. This paper first provides a brief literature review of works related   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to FGM fabrication, followed by reviewing works on employing machine learning   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in additive manufacturing, Afterward, we provide an overview of published works |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in the literature related to the application of machine learning methods in     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Directed Energy Deposition and for fabrication of FGMs.                         |                                   |                           |
| Universal Murray's law for optimised fluid transport in synthetic structures                                                                 | Binghan Zhou, Qian Cheng, Zhuo Chen, Zesheng Chen, Dongfang Liang, Eric Anthony Munro, Guolin Yun, Yoshiki Kawai, Jinrui Chen, Tynee Bhowmick, Padmanathan Karthick Kannan, Luigi G. Occhipinti, Hidetoshi Matsumoto, Julian Gardner, Bao-Lian Su, Tawfique Hasan                                                                                                                                                                                                                                                                                                                           | Materials following Murray's law are of significant interest due to their       | http://arxiv.org/abs/2309.16567v1 | 2023-09-28 16:24:16+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | unique porous structure and optimal mass transfer ability. However, it is       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | challenging to construct such biomimetic hierarchical channels with perfectly   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | cylindrical pores in synthetic systems following the existing theory. Achieving |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | superior mass transport capacity revealed by Murray's law in nanostructured     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | materials has thus far remained out of reach. We propose a Universal Murray's   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | law applicable to a wide range of hierarchical structures, shapes and           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | generalised transfer processes. We experimentally demonstrate optimal flow of   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | various fluids in hierarchically planar and tubular graphene aerogel structures |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to validate the proposed law. By adjusting the macroscopic pores in such        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | aerogel-based gas sensors, we also show a significantly improved sensor         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | response dynamic. Our work provides a solid framework for designing synthetic   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Murray materials with arbitrarily shaped channels for superior mass transfer    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | capabilities, with future implications in catalysis, sensing and energy         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications.                                                                   |                                   |                           |
| Influence of correlations in active medium on pump-induced exceptional points and strong coupling                                            | I. S. Pashkevich, I. V. Doronin, A. A. Zyablovsky, E. S. Andrianov                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Exceptional points show great prospects for applications such as imaging,       | http://arxiv.org/abs/2309.16566v1 | 2023-09-28 16:23:05+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | sensing and designing lasers. Recently, systems with amplifying active medium   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | exhibiting pump-induced exceptional points have attracted much attention due to |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | possibility of controlling strong coupling between light and matter with the    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | aid of pumping. In such structures, the interaction of active molecules with    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | external degrees of freedom, such as phonons or impurities, leads to the        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | destruction of the correlation between polarizations of different molecules. We |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | study the effect of the correlations decay on a system behavior near            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pump-induced exceptional point. We show that strong coupling persists and       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | eigenvectors together with eigenfrequencies coalesce at a negative value of     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | population inversion, regardless of correlation decay magnitude. Thus, we show  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | that exceptional points are robust to correlation decay, which is crucial for   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | designing systems with exceptional points.                                      |                                   |                           |
| SU(d)-Symmetric Random Unitaries: Quantum Scrambling, Error Correction, and Machine Learning                                                 | Zimu Li, Han Zheng, Yunfei Wang, Liang Jiang, Zi-Wen Liu, Junyu Liu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Quantum information processing in the presence of continuous symmetry is of     | http://arxiv.org/abs/2309.16556v1 | 2023-09-28 16:12:31+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | wide importance and exhibits many novel physical and mathematical phenomena.    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | SU(d) is a continuous symmetry group of particular interest since it represents |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | a fundamental type of non-Abelian symmetry and also plays a vital role in       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quantum computation. Here, we explicate the applications of SU(d)-symmetric     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | random unitaries in three different contexts ranging from physics to quantum    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | computing: information scrambling with non-Abelian conserved quantities,        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | covariant quantum error correcting random codes, and geometric quantum machine  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | learning. First, we show that, in the presence of SU(d) symmetry, the local     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | conserved quantities would exhibit residual values even at $t \rightarrow       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | \infty$ which decays as $\Omega(1/n^{3/2})$ under local Pauli basis for qubits  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and $\Omega(1/n^{(d+2)^2/2})$ under local symmetric basis for general qudits    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | with respect to the system size, in contrast to $O(1/n)$ decay for U(1) case    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and the exponential decay for no-symmetry case in the sense of out-of-time      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ordered correlator (OTOC). Second, we show that SU(d)-symmetric unitaries can   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | be used to construct asymptotically optimal (in the sense of saturating the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fundamental limits on the code error that have been called the approximate      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Eastin-Knill theorems) SU(d)-covariant codes that encodes any constant $k$      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | logical qudits, extending [Kong \& Liu; PRXQ 3, 020314 (2022)]. Finally, we     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | derive an overpartameterization threshold via the quantum neural tangent kernel |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (QNTK) required for exponential convergence guarantee of generic ansatz for     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | geometric quantum machine learning, which reveals that the number of parameters |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | required scales only with the dimension of desired subspaces rather than that   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of the entire Hilbert space. We expect that our work invites further research   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | on quantum information with continuous symmetries.                              |                                   |                           |
| MatrixCity: A Large-scale City Dataset for City-scale Neural Rendering and Beyond                                                            | Yixuan Li, Lihan Jiang, Linning Xu, Yuanbo Xiangli, Zhenzhi Wang, Dahua Lin, Bo Dai                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Neural radiance fields (NeRF) and its subsequent variants have led to           | http://arxiv.org/abs/2309.16553v1 | 2023-09-28 16:06:02+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | remarkable progress in neural rendering. While most of recent neural rendering  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | works focus on objects and small-scale scenes, developing neural rendering      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | methods for city-scale scenes is of great potential in many real-world          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications. However, this line of research is impeded by the absence of a     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | comprehensive and high-quality dataset, yet collecting such a dataset over real |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | city-scale scenes is costly, sensitive, and technically difficult. To this end, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | we build a large-scale, comprehensive, and high-quality synthetic dataset for   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | city-scale neural rendering researches. Leveraging the Unreal Engine 5 City     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Sample project, we develop a pipeline to easily collect aerial and street city  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | views, accompanied by ground-truth camera poses and a range of additional data  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | modalities. Flexible controls over environmental factors like light, weather,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | human and car crowd are also available in our pipeline, supporting the need of  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | various tasks covering city-scale neural rendering and beyond. The resulting    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pilot dataset, MatrixCity, contains 67k aerial images and 452k street images    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | from two city maps of total size $28km^2$. On top of MatrixCity, a thorough     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | benchmark is also conducted, which not only reveals unique challenges of the    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | task of city-scale neural rendering, but also highlights potential improvements |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | for future works. The dataset and code will be publicly available at our        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | project page: https://city-super.github.io/matrixcity/.                         |                                   |                           |
| Connection formulae for the radial Toda equations I                                                                                          | Martin A. Guest, Alexander R. Its, Maksim Kosmakov, Kenta Miyahara, Ryosuke Odoi                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | This paper is the first in a forthcoming series of works where the authors      | http://arxiv.org/abs/2309.16550v1 | 2023-09-28 16:02:12+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | study the global asymptotic behavior of the radial solutions of the 2D periodic |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Toda equation of type $A_n$. The principal issue is the connection formulae     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | between the asymptotic parameters describing the behavior of the general        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | solution at zero and infinity. To reach this goal we are using a fusion of the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | PDE analysis and the Riemann-Hilbert nonlinear steepest descent method of Deift |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and Zhou which is applicable to 2D Toda in view of its Lax integrability. A     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | principal technical challenge is the extension of the nonlinear steepest        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | descent analysis to Riemann-Hilbert problems of matrix rank greater than $2$.   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | In this paper, we meet this challenge for the case $n=2$ (the rank $3$ case)    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and it already captures the principal features of the general $n$ case.         |                                   |                           |
| Spin-resolved imaging of atomic-scale helimagnetism in monolayer NiI2                                                                        | Mao-Peng Miao, Nanshu Liu, Wen-Hao Zhang, Dao-Bo Wang, Wei Ji, Ying-Shuang Fu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | Identifying intrinsic noncollinear magnetic order in monolayer van der Waals    | http://arxiv.org/abs/2309.16526v1 | 2023-09-28 15:39:36+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (vdW) crystals is highly desirable for understanding the delicate magnetic      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | interactions at reduced spatial constraints and miniaturized spintronic         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications, but remains elusive in experiments. Here, we achieved             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | spin-resolved imaging of helimagnetism at atomic scale in monolayer NiI2        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | crystals, that were grown on graphene-covered SiC(0001) substrate, using        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | spin-polarized scanning tunneling microscopy. Our experiments identify the      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | existence of a spin spiral state with canted plane in monolayer NiI2. The spin  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | modulation Q vector of the spin spiral is determined as (0.2203, 0, 0), which   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | is different from its bulk value or its in-plane projection, but agrees well    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | with our first principles calculations. The spin spiral surprisingly indicates  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | collective spin switching behavior under magnetic field, whose origin is        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ascribed to the incommensurability between the spin spiral and the crystal      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | lattice. Our work unambiguously identifies the helimagnetic state in monolayer  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | NiI2, paving the way for illuminating its expected type-II multiferroic order   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and developing spintronic devices based on vdW magnets.                         |                                   |                           |
| Computationally efficient solution of mixed integer model predictive control problems via machine learning aided Benders Decomposition       | Ilias Mitrai, Prodromos Daoutidis                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | Mixed integer Model Predictive Control (MPC) problems arise in the operation    | http://arxiv.org/abs/2309.16508v1 | 2023-09-28 15:13:14+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of systems where discrete and continuous decisions must be taken simultaneously |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to compensate for disturbances. The efficient solution of mixed integer MPC     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problems requires the computationally efficient and robust online solution of   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | mixed integer optimization problems, which are generally difficult to solve. In |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | this paper, we propose a machine learning-based branch and check Generalized    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Benders Decomposition algorithm for the efficient solution of such problems. We |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | use machine learning to approximate the effect of the complicating variables on |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the subproblem by approximating the Benders cuts without solving the            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | subproblem, therefore, alleviating the need to solve the subproblem multiple    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | times. The proposed approach is applied to a mixed integer economic MPC case    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | study on the operation of chemical processes. We show that the proposed         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | algorithm always finds feasible solutions to the optimization problem, given    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | that the mixed integer MPC problem is feasible, and leads to a significant      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | reduction in solution time (up to 97% or 50x) while incurring small error (in   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the order of 1%) compared to the application of standard and accelerated        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Generalized Benders Decomposition.                                              |                                   |                           |
| Electronic and optical properties of 4H Si from first principles                                                                             | Xiao Zhang, Emmanouil Kioupakis                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | The cubic polytype of silicon (Si) is the most commercialized semiconductor     | http://arxiv.org/abs/2309.16501v1 | 2023-09-28 15:06:56+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | material and finds applications in numerous electronic and optoelectronic       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | devices such as solar cells. However, recent reports on the synthesis of the    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | hexagonal 4H Si polytype have attracted the attention of the scientific         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | community to understand its functional properties. Here we report the           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | electronic, vibrational, and optical properties of the 4H Si polytype obtained  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | with predictive first-principles calculations. We find that, compared to the    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | cubic polytype, 4H Si shows a slightly narrower indirect gap by $\sim$ 0.05 eV. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | By calculating its direct and phonon-assisted optical spectra, we show that 4H  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Si exhibits a stronger absorption coefficient than cubic Si across the visible  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and IR spectral regions. We further evaluate the short-circuit current density  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of textured thin-films, and we demonstrate that 4H Si can achieve the same      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | short-circuit current density for a five times thinner film compared to the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | cubic polytype. Our work demonstrates the advantages of 4H Si for thin-film     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | silicon-based solar-cell applications.                                          |                                   |                           |
| Asset Bundling for Wind Power Forecasting                                                                                                    | Hanyu Zhang, Mathieu Tanneau, Chaofan Huang, V. Roshan Joseph, Shangkun Wang, Pascal Van Hentenryck                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | The growing penetration of intermittent, renewable generation in US power       | http://arxiv.org/abs/2309.16492v1 | 2023-09-28 14:56:34+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | grids, especially wind and solar generation, results in increased operational   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | uncertainty. In that context, accurate forecasts are critical, especially for   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | wind generation, which exhibits large variability and is historically harder to |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | predict. To overcome this challenge, this work proposes a novel                 |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Bundle-Predict-Reconcile (BPR) framework that integrates asset bundling,        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | machine learning, and forecast reconciliation techniques. The BPR framework     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | first learns an intermediate hierarchy level (the bundles), then predicts wind  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | power at the asset, bundle, and fleet level, and finally reconciles all         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | forecasts to ensure consistency. This approach effectively introduces an        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | auxiliary learning task (predicting the bundle-level time series) to help the   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | main learning tasks. The paper also introduces new asset-bundling criteria that |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | capture the spatio-temporal dynamics of wind power time series. Extensive       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | numerical experiments are conducted on an industry-size dataset of 283 wind     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | farms in the MISO footprint. The experiments consider short-term and day-ahead  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | forecasts, and evaluates a large variety of forecasting models that include     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | weather predictions as covariates. The results demonstrate the benefits of BPR, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | which consistently and significantly improves forecast accuracy over baselines, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | especially at the fleet level.                                                  |                                   |                           |
| Multi-population Mean Field Games with Multiple Major Players: Application to Carbon Emission Regulations                                    | Gokce Dayanikli, Mathieu Lauriere                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | In this paper, we propose and study a mean field game model with multiple       | http://arxiv.org/abs/2309.16477v1 | 2023-09-28 14:41:04+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | populations of minor players and multiple major players, motivated by           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications to the regulation of carbon emissions. Each population of minor    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | players represent a large group of electricity producers and each major player  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | represents a regulator. We first characterize the minor players equilibrium     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | controls using forward-backward differential equations, and show existence and  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | uniqueness of the minor players equilibrium. We then express the major players' |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | equilibrium controls through analytical formulas given the other players'       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | controls. Finally, we then provide a method to solve the Nash equilibrium       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | between all the players, and we illustrate numerically the sensitivity of the   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | model to its parameters.                                                        |                                   |                           |
| Circuit-to-Hamiltonian from tensor networks and fault tolerance                                                                              | Anurag Anshu, Nikolas P. Breuckmann, Quynh T. Nguyen                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | We define a map from an arbitrary quantum circuit to a local Hamiltonian        | http://arxiv.org/abs/2309.16475v1 | 2023-09-28 14:39:13+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | whose ground state encodes the quantum computation. All previous maps relied on |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the Feynman-Kitaev construction, which introduces an ancillary `clock register' |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to track the computational steps. Our construction, on the other hand, relies   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | on injective tensor networks with associated parent Hamiltonians, avoiding the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | introduction of a clock register. This comes at the cost of the ground state    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | containing only a noisy version of the quantum computation, with independent    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | stochastic noise. We can remedy this - making our construction robust - by      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | using quantum fault tolerance. In addition to the stochastic noise, we show     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | that any state with energy density exponentially small in the circuit depth     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | encodes a noisy version of the quantum computation with adversarial noise. We   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | also show that any `combinatorial state' with energy density polynomially small |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in depth encodes the quantum computation with adversarial noise. This serves as |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | evidence that any state with energy density polynomially small in depth has a   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | similar property. As an application, we show that contracting injective tensor  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | networks to additive error is BQP-hard. We also discuss the implication of our  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | construction to the quantum PCP conjecture, combining with an observation that  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | QMA verification can be done in logarithmic depth.                              |                                   |                           |
| Hadoop-Oriented SVM-LRU (H-SVM-LRU): An Intelligent Cache Replacement Algorithm to Improve MapReduce Performance                             | Rana Ghazali, Sahar Adabi, Ali Rezaee, Douglas G. Down, Ali Movaghar                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | Modern applications can generate a large amount of data from different          | http://arxiv.org/abs/2309.16471v1 | 2023-09-28 14:36:38+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | sources with high velocity, a combination that is difficult to store and        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | process via traditional tools. Hadoop is one framework that is used for the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | parallel processing of a large amount of data in a distributed environment,     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | however, various challenges can lead to poor performance. Two particular issues |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | that can limit performance are the high access time for I/O operations and the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | recomputation of intermediate data. The combination of these two issues can     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | result in resource wastage. In recent years, there have been attempts to        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | overcome these problems by using caching mechanisms. Due to cache space         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | limitations, it is crucial to use this space efficiently and avoid cache        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pollution (the cache contains data that is not used in the future). We propose  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Hadoop-oriented SVM-LRU (HSVM- LRU) to improve Hadoop performance. For this     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | purpose, we use an intelligent cache replacement algorithm, SVM-LRU, that       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | combines the well-known LRU mechanism with a machine learning algorithm, SVM,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to classify cached data into two groups based on their future usage.            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Experimental results show a significant decrease in execution time as a result  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of an increased cache hit ratio, leading to a positive impact on Hadoop         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance.                                                                    |                                   |                           |
| HyperLISTA-ABT: An Ultra-light Unfolded Network for Accurate Multi-component Differential Tomographic SAR Inversion                          | Kun Qian, Yuanyuan Wang, Peter Jung, Yilei Shi, Xiao Xiang Zhu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | Deep neural networks based on unrolled iterative algorithms have achieved       | http://arxiv.org/abs/2309.16468v1 | 2023-09-28 14:34:21+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | remarkable success in sparse reconstruction applications, such as synthetic     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | aperture radar (SAR) tomographic inversion (TomoSAR). However, the currently    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | available deep learning-based TomoSAR algorithms are limited to                 |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | three-dimensional (3D) reconstruction. The extension of deep learning-based     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | algorithms to four-dimensional (4D) imaging, i.e., differential TomoSAR         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (D-TomoSAR) applications, is impeded mainly due to the high-dimensional weight  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | matrices required by the network designed for D-TomoSAR inversion, which        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | typically contain millions of freely trainable parameters. Learning such huge   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | number of weights requires an enormous number of training samples, resulting in |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | a large memory burden and excessive time consumption. To tackle this issue, we  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | propose an efficient and accurate algorithm called HyperLISTA-ABT. The weights  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in HyperLISTA-ABT are determined in an analytical way according to a minimum    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | coherence criterion, trimming the model down to an ultra-light one with only    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | three hyperparameters. Additionally, HyperLISTA-ABT improves the global         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | thresholding by utilizing an adaptive blockwise thresholding scheme, which      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applies block-coordinate techniques and conducts thresholding in local blocks,  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | so that weak expressions and local features can be retained in the shrinkage    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | step layer by layer. Simulations were performed and demonstrated the            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | effectiveness of our approach, showing that HyperLISTA-ABT achieves superior    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | computational efficiency and with no significant performance degradation        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | compared to state-of-the-art methods. Real data experiments showed that a       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | high-quality 4D point cloud could be reconstructed over a large area by the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | proposed HyperLISTA-ABT with affordable computational resources and in a fast   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | time.                                                                           |                                   |                           |
| Augmenting LLMs with Knowledge: A survey on hallucination prevention                                                                         | Konstantinos Andriopoulos, Johan Pouwelse                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | Large pre-trained language models have demonstrated their proficiency in        | http://arxiv.org/abs/2309.16459v1 | 2023-09-28 14:09:58+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | storing factual knowledge within their parameters and achieving remarkable      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | results when fine-tuned for downstream natural language processing tasks.       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Nonetheless, their capacity to access and manipulate knowledge with precision   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | remains constrained, resulting in performance disparities on                    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | knowledge-intensive tasks when compared to task-specific architectures.         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Additionally, the challenges of providing provenance for model decisions and    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | maintaining up-to-date world knowledge persist as open research frontiers. To   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | address these limitations, the integration of pre-trained models with           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | differentiable access mechanisms to explicit non-parametric memory emerges as a |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | promising solution. This survey delves into the realm of language models (LMs)  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | augmented with the ability to tap into external knowledge sources, including    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | external knowledge bases and search engines. While adhering to the standard     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | objective of predicting missing tokens, these augmented LMs leverage diverse,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | possibly non-parametric external modules to augment their contextual processing |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | capabilities, departing from the conventional language modeling paradigm.       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Through an exploration of current advancements in augmenting large language     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | models with knowledge, this work concludes that this emerging research          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | direction holds the potential to address prevalent issues in traditional LMs,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | such as hallucinations, un-grounded responses, and scalability challenges.      |                                   |                           |
| Novel Discrete Composite Distributions with Applications to Infectious Disease Data                                                          | Bowen Liu, Malwane M. A. Ananda                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | It was observed that the number of cases and deaths for infectious diseases     | http://arxiv.org/abs/2309.16443v1 | 2023-09-28 13:53:54+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | were associated with heavy-tailed power law distributions such as the Pareto    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | distribution. While Pareto distribution was widely used to model the cases and  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | deaths of infectious diseases, a major limitation of Pareto distribution is     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | that it can only fit a given data set beyond a certain threshold. Thus, it can  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | only model part of the data set. Thus, we proposed some novel discrete          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | composite distributions with Pareto tails to fit the real infectious disease    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | data. To provide necessary statistical inference for the tail behavior of the   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | data, we developed a hypothesis testing procedure to test the tail index        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | parameter. COVID-19 reported cases in Singapore and monkeypox reported cases in |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | France were analyzed to evaluate the performance of the newly created           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | distributions. The results from the analysis suggested that the discrete        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | composite distributions could demonstrate competitive performance compared to   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the commonly used discrete distributions. Furthermore, the analysis of the tail |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | index parameter can provide great insights into preventing and controlling      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | infectious diseases.                                                            |                                   |                           |
| Uncertainty quantification and complex analyticity of the nonlinear Poisson-Boltzmann equation for the interface problem with random domains | Trevor Norton, Jie Xu, Brian Choi, Mark Kon, Julio Enrique Castrillón-Candás                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | The nonlinear Poisson-Boltzmann equation (NPBE) is an elliptic partial          | http://arxiv.org/abs/2309.16439v1 | 2023-09-28 13:46:57+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | differential equation used in applications such as protein interactions and     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | biophysical chemistry (among many others). It describes the nonlinear           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | electrostatic potential of charged bodies submerged in an ionic solution. The   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | kinetic presence of the solvent molecules introduces randomness to the shape of |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | a protein, and thus a more accurate model that incorporates these random        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | perturbations of the domain is analyzed to compute the statistics of quantities |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of interest of the solution. When the parameterization of the random            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | perturbations is high-dimensional, this calculation is intractable as it is     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | subject to the curse of dimensionality. However, if the solution of the NPBE    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | varies analytically with respect to the random parameters, the problem becomes  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | amenable to techniques such as sparse grids and deep neural networks. In this   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | paper, we show analyticity of the solution of the NPBE with respect to analytic |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | perturbations of the domain by using the analytic implicit function theorem and |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the domain mapping method. Previous works have shown analyticity of solutions   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to linear elliptic equations but not for nonlinear problems. We further show    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | how to derive \emph{a priori} bounds on the size of the region of analyticity.  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | This method is applied to the trypsin molecule to demonstrate that the          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | convergence rates of the quantity of interest are consistent with the           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | analyticity result. Furthermore, the approach developed here is sufficiently    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | general enough to be applied to other nonlinear problems in uncertainty         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quantification.                                                                 |                                   |                           |
| Neuro Symbolic Reasoning for Planning: Counterexample Guided Inductive Synthesis using Large Language Models and Satisfiability Solving      | Sumit Kumar Jha, Susmit Jha, Patrick Lincoln, Nathaniel D. Bastian, Alvaro Velasquez, Rickard Ewetz, Sandeep Neema                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Generative large language models (LLMs) with instruct training such as GPT-4    | http://arxiv.org/abs/2309.16436v1 | 2023-09-28 13:40:50+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | can follow human-provided instruction prompts and generate human-like responses |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to these prompts. Apart from natural language responses, they have also been    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | found to be effective at generating formal artifacts such as code, plans, and   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | logical specifications from natural language prompts. Despite their remarkably  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | improved accuracy, these models are still known to produce factually incorrect  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | or contextually inappropriate results despite their syntactic coherence - a     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | phenomenon often referred to as hallucination. This limitation makes it         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | difficult to use these models to synthesize formal artifacts that are used in   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | safety-critical applications. Unlike tasks such as text summarization and       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | question-answering, bugs in code, plan, and other formal artifacts produced by  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | LLMs can be catastrophic. We posit that we can use the satisfiability modulo    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | theory (SMT) solvers as deductive reasoning engines to analyze the generated    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | solutions from the LLMs, produce counterexamples when the solutions are         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | incorrect, and provide that feedback to the LLMs exploiting the dialog          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | capability of instruct-trained LLMs. This interaction between inductive LLMs    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and deductive SMT solvers can iteratively steer the LLM to generate the correct |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | response. In our experiments, we use planning over the domain of blocks as our  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | synthesis task for evaluating our approach. We use GPT-4, GPT3.5 Turbo,         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Davinci, Curie, Babbage, and Ada as the LLMs and Z3 as the SMT solver. Our      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | method allows the user to communicate the planning problem in natural language; |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | even the formulation of queries to SMT solvers is automatically generated from  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | natural language. Thus, the proposed technique can enable non-expert users to   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | describe their problems in natural language, and the combination of LLMs and    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | SMT solvers can produce provably correct solutions.                             |                                   |                           |
| On methods for assessment of the influence and impact of observations in convection-permitting numerical weather prediction                  | Guannan Hu, Sarah L. Dance, Alison Fowler, David Simonin, Joanne Waller, Thomas Auligne, Sean Healy, Daisuke Hotta, Ulrich Löhnert, Takemasa Miyoshi, Nikki C. Prive, Olaf Stiller, Xuguang Wang, Martin Weissmann                                                                                                                                                                                                                                                                                                                                                                          | In numerical weather prediction (NWP), a large number of observations are       | http://arxiv.org/abs/2309.16433v1 | 2023-09-28 13:33:06+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | used to create initial conditions for weather forecasting through a process     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | known as data assimilation. An assessment of the value of these observations    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | for NWP can guide us in the design of future observation networks, help us to   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | identify problems with the assimilation system, and allow us to assess changes  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to the assimilation system. However, the assessment can be challenging in       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | convection-permitting NWP. First, the strong nonlinearity in the forecast model |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | limits the methods available for the assessment. Second, convection-permitting  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | NWP typically uses a limited area model and provides short forecasts, giving    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problems with verification and our ability to gather sufficient statistics.     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Third, convection-permitting NWP often makes use of novel observations, which   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | can be difficult to simulate in an observing system simulation experiment       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (OSSE). We compare methods that can be used to assess the value of observations |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in convection-permitting NWP and discuss operational considerations when using  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | these methods. We focus on their applicability to ensemble forecasting systems, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | as these systems are becoming increasingly dominant for convection-permitting   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | NWP. We also identify several future research directions: comparison of         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | forecast validation using analyses and observations, the effect of ensemble     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | size on assessing the value of observations, flow-dependent covariance          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | localization, and generation and validation of the nature run in an OSSE.       |                                   |                           |
| Nonlinear MPC design for incrementally ISS systems with application to GRU networks                                                          | Fabio Bonassi, Alessio La Bella, Marcello Farina, Riccardo Scattolini                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | This brief addresses the design of a Nonlinear Model Predictive Control         | http://arxiv.org/abs/2309.16428v1 | 2023-09-28 13:26:20+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (NMPC) strategy for exponentially incremental Input-to-State Stable (ISS)       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | systems. In particular, a novel formulation is devised, which does not          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | necessitate the onerous computation of terminal ingredients, but rather relies  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | on the explicit definition of a minimum prediction horizon ensuring closed-loop |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | stability. The designed methodology is particularly suited for the control of   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | systems learned by Recurrent Neural Networks (RNNs), which are known for their  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | enhanced modeling capabilities and for which the incremental ISS properties can |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | be studied thanks to simple algebraic conditions. The approach is applied to    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Gated Recurrent Unit (GRU) networks, providing also a method for the design of  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | a tailored state observer with convergence guarantees. The resulting control    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | architecture is tested on a benchmark system, demonstrating its good control    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performances and efficient applicability.                                       |                                   |                           |
| Distilling ODE Solvers of Diffusion Models into Smaller Steps                                                                                | Sanghwan Kim, Hao Tang, Fisher Yu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | Distillation techniques have substantially improved the sampling speed of       | http://arxiv.org/abs/2309.16421v1 | 2023-09-28 13:12:18+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | diffusion models, allowing of the generation within only one step or a few      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | steps. However, these distillation methods require extensive training for each  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | dataset, sampler, and network, which limits their practical applicability. To   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | address this limitation, we propose a straightforward distillation approach,    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Distilled-ODE solvers (D-ODE solvers), that optimizes the ODE solver rather     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | than training the denoising network. D-ODE solvers are formulated by simply     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applying a single parameter adjustment to existing ODE solvers. Subsequently,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | D-ODE solvers with smaller steps are optimized by ODE solvers with larger steps |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | through distillation over a batch of samples. Our comprehensive experiments     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | indicate that D-ODE solvers outperform existing ODE solvers, including DDIM,    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | PNDM, DPM-Solver, DEIS, and EDM, especially when generating samples with fewer  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | steps. Our method incur negligible computational overhead compared to previous  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | distillation techniques, enabling simple and rapid integration with previous    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | samplers. Qualitative analysis further shows that D-ODE solvers enhance image   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quality while preserving the sampling trajectory of ODE solvers.                |                                   |                           |
| Selective Nonparametric Regression via Testing                                                                                               | Fedor Noskov, Alexander Fishkov, Maxim Panov                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | Prediction with the possibility of abstention (or selective prediction) is an   | http://arxiv.org/abs/2309.16412v1 | 2023-09-28 13:04:11+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | important problem for error-critical machine learning applications. While       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | well-studied in the classification setup, selective approaches to regression    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | are much less developed. In this work, we consider the nonparametric            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | heteroskedastic regression problem and develop an abstention procedure via      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | testing the hypothesis on the value of the conditional variance at a given      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | point. Unlike existing methods, the proposed one allows to account not only for |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the value of the variance itself but also for the uncertainty of the            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | corresponding variance predictor. We prove non-asymptotic bounds on the risk of |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the resulting estimator and show the existence of several different convergence |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | regimes. Theoretical analysis is illustrated with a series of experiments on    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | simulated and real-world data.                                                  |                                   |                           |
| Empirically Driven Spline Bases for Functional Principal Component Analysis in 2D                                                            | Rani Basna, Hiba Nassar, Krzysztof Podgórski                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | Functional data analysis is typically performed in two steps: first,            | http://arxiv.org/abs/2309.16402v1 | 2023-09-28 12:48:53+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | functionally representing discrete observations, and then applying functional   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | methods to the so-represented data. The initial choice of a functional          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | representation may have a significant impact on the second phase of the         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | analysis, as shown in recent research, where data-driven spline bases           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | outperformed the predefined rigid choice of functional representation. The      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | method chooses an initial functional basis by an efficient placement of the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | knots using a simple machine-learning algorithm. The approach does not apply    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | directly when the data are defined on domains of a higher dimension than one    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | such as, for example, images. The reason is that in higher dimensions the       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | convenient and numerically efficient spline bases are obtained as tensor bases  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | from 1D spline bases that require knots that are located on a lattice. This     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | does not allow for a flexible knot placement that was fundamental for the 1D    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | approach. The goal of this research is to propose two modified approaches that  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | circumvent the problem by coding the irregular knot selection into their        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | densities and utilizing these densities through the topology of the spaces of   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | splines. This allows for regular grids for the knots and thus facilitates using |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the spline tensor bases. It is tested on 1D data showing that its performance   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | is comparable to or better than the previous methods.                           |                                   |                           |
| Recent Advances of Differential Privacy in Centralized Deep Learning: A Systematic Survey                                                    | Lea Demelius, Roman Kern, Andreas Trügler                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | Differential Privacy has become a widely popular method for data protection     | http://arxiv.org/abs/2309.16398v1 | 2023-09-28 12:44:59+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in machine learning, especially since it allows formulating strict mathematical |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | privacy guarantees. This survey provides an overview of the state-of-the-art of |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | differentially private centralized deep learning, thorough analyses of recent   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | advances and open problems, as well as a discussion of potential future         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | developments in the field. Based on a systematic literature review, the         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | following topics are addressed: auditing and evaluation methods for private     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | models, improvements of privacy-utility trade-offs, protection against a broad  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | range of threats and attacks, differentially private generative models, and     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | emerging application domains.                                                   |                                   |                           |
| A Comprehensive Survey of Document-level Relation Extraction (2016-2022)                                                                     | Julien Delaunay, Thi Hong Hanh Tran, Carlos-Emiliano González-Gallardo, Georgeta Bordea, Nicolas Sidere, Antoine Doucet                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | Document-level relation extraction (DocRE) is an active area of research in     | http://arxiv.org/abs/2309.16396v1 | 2023-09-28 12:43:32+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | natural language processing (NLP) concerned with identifying and extracting     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | relationships between entities beyond sentence boundaries. Compared to the more |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | traditional sentence-level relation extraction, DocRE provides a broader        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | context for analysis and is more challenging because it involves identifying    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | relationships that may span multiple sentences or paragraphs. This task has     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | gained increased interest as a viable solution to build and populate knowledge  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | bases automatically from unstructured large-scale documents (e.g., scientific   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | papers, legal contracts, or news articles), in order to have a better           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | understanding of relationships between entities. This paper aims to provide a   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | comprehensive overview of recent advances in this field, highlighting its       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | different applications in comparison to sentence-level relation extraction.     |                                   |                           |
| Streaming quantum state purification                                                                                                         | Andrew M. Childs, Honghao Fu, Debbie Leung, Zhi Li, Maris Ozols, Vedang Vyas                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | Quantum state purification is the task of recovering a nearly pure copy of an   | http://arxiv.org/abs/2309.16387v1 | 2023-09-28 12:33:09+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | unknown pure quantum state using multiple noisy copies of the state. This basic |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | task has applications to quantum communication over noisy channels and quantum  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | computation with imperfect devices, but has only been studied previously for    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the case of qubits. We derive an efficient purification procedure based on the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | swap test for qudits of any dimension, starting with any initial error          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | parameter. Treating the initial error parameter and the dimension as constants, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | we show that our procedure has sample complexity asymptotically optimal in the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | final error parameter. Our protocol has a simple recursive structure that can   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | be applied when the states are provided one at a time in a streaming fashion,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | requiring only a small quantum memory to implement.                             |                                   |                           |
| RLLTE: Long-Term Evolution Project of Reinforcement Learning                                                                                 | Mingqi Yuan, Zequn Zhang, Yang Xu, Shihao Luo, Bo Li, Xin Jin, Wenjun Zeng                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | We present RLLTE: a long-term evolution, extremely modular, and open-source     | http://arxiv.org/abs/2309.16382v1 | 2023-09-28 12:30:37+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | framework for reinforcement learning (RL) research and application. Beyond      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | delivering top-notch algorithm implementations, RLLTE also serves as a toolkit  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | for developing algorithms. More specifically, RLLTE decouples the RL algorithms |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | completely from the exploitation-exploration perspective, providing a large     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | number of components to accelerate algorithm development and evolution. In      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | particular, RLLTE is the first RL framework to build a complete and luxuriant   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ecosystem, which includes model training, evaluation, deployment, benchmark     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | hub, and large language model (LLM)-empowered copilot. RLLTE is expected to set |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | standards for RL engineering practice and be highly stimulative for industry    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and academia.                                                                   |                                   |                           |
| Nek5000/RS Performance on Advanced GPU Architectures                                                                                         | Misun Min, Yu-Hsiang Lan, Paul Fischer, Thilina Rathnayake, John Holmen                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | We demonstrate NekRS performance results on various advanced GPU                | http://arxiv.org/abs/2309.16381v1 | 2023-09-28 12:29:02+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | architectures. NekRS is a GPU-accelerated version of Nek5000 that targets high  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance on exascale platforms. It is being developed in DOE's Center of     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Efficient Exascale Discretizations, which is one of the co-design centers under |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the Exascale Computing Project. In this paper, we consider Frontier, Crusher,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Spock, Polaris, Perlmutter, ThetaGPU, and Summit. Simulations are performed     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | with 17x17 rod-bundle geometries from small modular reactor applications. We    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | discuss strong-scaling performance and analysis.                                |                                   |                           |
| A microlocal investigation of stochastic partial differential equations for spinors with an application to the Thirring model                | Alberto Bonicelli, Beatrice Costeri, Claudio Dappiaggi, Paolo Rinaldi                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | On a $d$-dimensional Riemannian, spin manifold $(M,g)$ we consider              | http://arxiv.org/abs/2309.16376v1 | 2023-09-28 12:24:06+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | non-linear, stochastic partial differential equations for spinor fields, driven |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | by a Dirac operator and coupled to an additive Gaussian, vector-valued white    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | noise. We extend to the case in hand a procedure, introduced in [DDRZ20] for    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the scalar counterpart, which allows to compute at a perturbative level the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | expectation value of the solutions as well as the associated correlation        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | functions accounting intrinsically for the underlying renormalization freedoms. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | This framework relies strongly on tools proper of microlocal analysis and it is |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | inspired by the algebraic approach to quantum field theory. As a concrete       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | example we apply it to a stochastic version of the Thirring model proving in    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | particular that it lies in the subcritical regime if $d\leq 2$.                 |                                   |                           |
| A Comprehensive Review on Tree Detection Methods Using Point Cloud and Aerial Imagery from Unmanned Aerial Vehicles                          | Weijie Kuang, Hann Woei Ho, Ye Zhou, Shahrel Azmin Suandi, Farzad Ismail                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Unmanned Aerial Vehicles (UAVs) are considered cutting-edge technology with     | http://arxiv.org/abs/2309.16375v1 | 2023-09-28 12:22:39+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | highly cost-effective and flexible usage scenarios. Although many papers have   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | reviewed the application of UAVs in agriculture, the review of the application  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | for tree detection is still insufficient. This paper focuses on tree detection  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | methods applied to UAV data collected by UAVs. There are two kinds of data, the |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | point cloud and the images, which are acquired by the Light Detection and       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Ranging (LiDAR) sensor and camera, respectively. Among the detection methods    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | using point-cloud data, this paper mainly classifies these methods according to |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | LiDAR and Digital Aerial Photography (DAP). For the detection methods using     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | images directly, this paper reviews these methods by whether or not to use the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Deep Learning (DL) method. Our review concludes and analyses the comparison and |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | combination between the application of LiDAR-based and DAP-based point cloud    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | data. The performance, relative merits, and application fields of the methods   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | are also introduced. Meanwhile, this review counts the number of tree detection |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | studies using different methods in recent years. From our statics, the          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | detection task using DL methods on the image has become a mainstream trend as   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the number of DL-based detection researches increases to 45% of the total       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | number of tree detection studies up to 2022. As a result, this review could     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | help and guide researchers who want to carry out tree detection on specific     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | forests and for farmers to use UAVs in managing agriculture production.         |                                   |                           |
| Regularization and Model Selection for Item-on-Items Regression with Applications to Food Products' Survey Data                              | Aisouda Hoshiyar, Laura H. Gertheiss, Jan Gertheiss                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Ordinal data are quite common in applied statistics. Although some model        | http://arxiv.org/abs/2309.16373v1 | 2023-09-28 12:18:02+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | selection and regularization techniques for categorical predictors and ordinal  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | response models have been developed over the past few years, less work has been |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | done concerning ordinal-on-ordinal regression. Motivated by survey datasets on  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | food products consisting of Likert-type items, we propose a strategy for        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | smoothing and selection of ordinally scaled predictors in the cumulative logit  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | model. First, the original group lasso is modified by use of difference         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | penalties on neighbouring dummy coefficients, thus taking into account the      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | predictors' ordinal structure. Second, a fused lasso type penalty is presented  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | for fusion of predictor categories and factor selection. The performance of     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | both approaches is evaluated in simulation studies, while our primary case      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | study is a survey on the willingness to pay for luxury food products.           |                                   |                           |
| Application of Rigorous Interface Boundary Conditions in Mesoscale Plasticity Simulations                                                    | Jinxin Yu, Alfonso H. W. Ngan, David J. Srolovitz, Jian Han                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | The interactions between dislocations and interface/grain boundaries,           | http://arxiv.org/abs/2309.16366v1 | 2023-09-28 12:09:01+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | including dislocation absorption, transmission, and reflection, have garnered   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | significant attention from the research community for their impact on the       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | mechanical properties of materials. However, the traditional approaches used to |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | simulate grain boundaries lack physical fidelity and are often incompatible     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | across different simulation methods. We review a new mesoscale interface        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | boundary condition based on Burgers vector conservation and kinetic dislocation |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | reaction processes. The main focus of the paper is to demonstrate how to unify  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | this boundary condition with different plasticity simulation approaches such as |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the crystal plasticity finite element, continuum dislocation dynamics, and      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | discrete dislocation dynamics methods. To validate our interface boundary       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | condition, we implemented simulations using both the crystal plasticity finite  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | element method and a two-dimensional continuum dislocation dynamics model. Our  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | results show that our compact and physically realistic interface boundary       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | condition can be easily integrated into multiscale simulation methods and yield |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | novel results consistent with experimental observations.                        |                                   |                           |
| Libertas: Privacy-Preserving Computation for Decentralised Personal Data Stores                                                              | Rui Zhao, Naman Goel, Nitin Agrawal, Jun Zhao, Jake Stein, Ruben Verborgh, Reuben Binns, Tim Berners-Lee, Nigel Shadbolt                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Data-driven decision-making and AI applications present exciting new            | http://arxiv.org/abs/2309.16365v1 | 2023-09-28 12:07:40+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | opportunities delivering widespread benefits. The rapid adoption of such        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications triggers legitimate concerns about loss of privacy and misuse of   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | personal data. This leads to a growing and pervasive tension between harvesting |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ubiquitous data on the Web and the need to protect individuals. Decentralised   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | personal data stores (PDS) such as Solid are frameworks designed to give        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | individuals ultimate control over their personal data. But current PDS          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | approaches have limited support for ensuring privacy when computations combine  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | data spread across users. Secure Multi-Party Computation (MPC) is a well-known  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | subfield of cryptography, enabling multiple autonomous parties to               |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | collaboratively compute a function while ensuring the secrecy of inputs (input  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | privacy). These two technologies complement each other, but existing practices  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fall short in addressing the requirements and challenges of introducing MPC in  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | a PDS environment. For the first time, we propose a modular design for          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | integrating MPC with Solid while respecting the requirements of                 |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | decentralisation in this context. Our architecture, Libertas, requires no       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | protocol level changes in the underlying design of Solid, and can be adapted to |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | other PDS. We further show how this can be combined with existing differential  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | privacy techniques to also ensure output privacy. We use empirical benchmarks   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to inform and evaluate our implementation and design choices. We show the       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | technical feasibility and scalability pattern of the proposed system in two     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | novel scenarios -- 1) empowering gig workers with aggregate computations on     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | their earnings data; and 2) generating high-quality differentially-private      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | synthetic data without requiring a trusted centre. With this, we demonstrate    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the linear scalability of Libertas, and gained insights about compute           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | optimisations under such an architecture.                                       |                                   |                           |
| Integrating quantum and classical computing for multi-energy system optimization using Benders decomposition                                 | Ludger Leenders, Martin Sollich, Christiane Reinert, André Bardow                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | During recent years, quantum computers have received increasing attention,      | http://arxiv.org/abs/2309.16363v1 | 2023-09-28 11:59:42+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | primarily due to their ability to significantly increase computational          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance for specific problems. Computational performance could be improved  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | for mathematical optimization by quantum annealers. This special type of        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quantum computer can solve quadratic unconstrained binary optimization          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problems. However, multi-energy systems optimization commonly involves integer  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and continuous decision variables. Due to their mixed-integer problem           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | structure, quantum annealers cannot be directly used for multi-energy system    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | optimization. To solve multi-energy system optimization problems, we present a  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | hybrid Benders decomposition approach combining optimization on quantum and     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | classical computers. In our approach, the quantum computer solves the master    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problem, which involves only the integer variables from the original energy     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | system optimization problem. The subproblem includes the continuous variables   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and is solved by a classical computer. For better performance, we apply         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | improvement techniques to the Benders decomposition. We test the approach on a  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | case study to design a cost-optimal multi-energy system. While we provide a     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | proof of concept that our Benders decomposition approach is applicable for the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | design of multi-energy systems, the computational time is still higher than for |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | approaches using classical computers only. We therefore estimate the potential  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | improvement of our approach to be expected for larger and fault-tolerant        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quantum computers.                                                              |                                   |                           |
| Smoothing the Nonsmoothness                                                                                                                  | Chaohua Dong, Jiti Gao, Bin Peng, Yundong Tu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | To tackle difficulties for theoretical studies in situations involving          | http://arxiv.org/abs/2309.16348v1 | 2023-09-28 11:14:55+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | nonsmooth functions, we propose a sequence of infinitely differentiable         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | functions to approximate the nonsmooth function under consideration. A rate of  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | approximation is established and an illustration of its application is then     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | provided.                                                                       |                                   |                           |
| Intrinsic Language-Guided Exploration for Complex Long-Horizon Robotic Manipulation Tasks                                                    | Eleftherios Triantafyllidis, Filippos Christianos, Zhibin Li                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | Current reinforcement learning algorithms struggle in sparse and complex        | http://arxiv.org/abs/2309.16347v1 | 2023-09-28 11:14:52+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | environments, most notably in long-horizon manipulation tasks entailing a       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | plethora of different sequences. In this work, we propose the Intrinsically     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Guided Exploration from Large Language Models (IGE-LLMs) framework. By          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | leveraging LLMs as an assistive intrinsic reward, IGE-LLMs guides the           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | exploratory process in reinforcement learning to address intricate long-horizon |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | with sparse rewards robotic manipulation tasks. We evaluate our framework and   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | related intrinsic learning methods in an environment challenged with            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | exploration, and a complex robotic manipulation task challenged by both         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | exploration and long-horizons. Results show IGE-LLMs (i) exhibit notably higher |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance over related intrinsic methods and the direct use of LLMs in        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | decision-making, (ii) can be combined and complement existing learning methods  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | highlighting its modularity, (iii) are fairly insensitive to different          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | intrinsic scaling parameters, and (iv) maintain robustness against increased    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | levels of uncertainty and horizons.                                             |                                   |                           |
| Epistemic Logic Programs: a study of some properties                                                                                         | Stefania Costantini, Andrea Formisano                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Epistemic Logic Programs (ELPs), extend Answer Set Programming (ASP) with       | http://arxiv.org/abs/2309.16344v1 | 2023-09-28 11:08:37+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | epistemic operators. The semantics of such programs is provided in terms of     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | world views, which are sets of belief sets, i.e., syntactically, sets of sets   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of atoms. Different semantic approaches propose different characterizations of  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | world views. Recent work has introduced semantic properties that should be met  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | by any semantics for ELPs, like the Epistemic Splitting Property, that, if      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | satisfied, allows to modularly compute world views in a bottom-up fashion,      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | analogously to ``traditional'' ASP. We analyze the possibility of changing the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | perspective, shifting from a bottom-up to a top-down approach to splitting. We  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | propose a basic top-down approach, which we prove to be equivalent to the       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | bottom-up one. We then propose an extended approach, where our new definition:  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (i) is provably applicable to many of the existing semantics; (ii) operates     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | similarly to ``traditional'' ASP; (iii) provably coincides under any semantics  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | with the bottom-up notion of splitting at least on the class of Epistemically   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Stratified Programs (which are, intuitively, those where the use of epistemic   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | operators is stratified); (iv) better adheres to common ASP programming         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | methodology.                                                                    |                                   |                           |
| LagrangeBench: A Lagrangian Fluid Mechanics Benchmarking Suite                                                                               | Artur P. Toshev, Gianluca Galletti, Fabian Fritz, Stefan Adami, Nikolaus A. Adams                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | Machine learning has been successfully applied to grid-based PDE modeling in    | http://arxiv.org/abs/2309.16342v1 | 2023-09-28 11:03:23+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | various scientific applications. However, learned PDE solvers based on          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Lagrangian particle discretizations, which are the preferred approach to        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problems with free surfaces or complex physics, remain largely unexplored. We   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | present LagrangeBench, the first benchmarking suite for Lagrangian particle     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problems, focusing on temporal coarse-graining. In particular, our contribution |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | is: (a) seven new fluid mechanics datasets (four in 2D and three in 3D)         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | generated with the Smoothed Particle Hydrodynamics (SPH) method including the   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Taylor-Green vortex, lid-driven cavity, reverse Poiseuille flow, and dam break, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | each of which includes different physics like solid wall interactions or free   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | surface, (b) efficient JAX-based API with various recent training strategies    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and neighbors search routine, and (c) JAX implementation of established Graph   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Neural Networks (GNNs) like GNS and SEGNN with baseline results. Finally, to    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | measure the performance of learned surrogates we go beyond established position |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | errors and introduce physical metrics like kinetic energy MSE and Sinkhorn      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | distance for the particle distribution. Our codebase is available under the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | URL: https://github.com/tumaer/lagrangebench                                    |                                   |                           |
| EFFL: Egalitarian Fairness in Federated Learning for Mitigating Matthew Effect                                                               | Jiashi Gao, Changwu Huang, Ming Tang, Shin Hwei Tan, Xin Yao, Xuetao Wei                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Recent advances in federated learning (FL) enable collaborative training of     | http://arxiv.org/abs/2309.16338v1 | 2023-09-28 10:51:12+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | machine learning (ML) models from large-scale and widely dispersed clients      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | while protecting their privacy. However, when different clients' datasets are   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | heterogeneous, traditional FL mechanisms produce a global model that does not   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | adequately represent the poorer clients with limited data resources, resulting  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in lower accuracy and higher bias on their local data. According to the Matthew |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | effect, which describes how the advantaged gain more advantage and the          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | disadvantaged lose more over time, deploying such a global model in client      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications may worsen the resource disparity among the clients and harm the   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | principles of social welfare and fairness. To mitigate the Matthew effect, we   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | propose Egalitarian Fairness Federated Learning (EFFL), where egalitarian       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fairness refers to the global model learned from FL has: (1) equal accuracy     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | among clients; (2) equal decision bias among clients. Besides achieving         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | egalitarian fairness among the clients, EFFL also aims for performance          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | optimality, minimizing the empirical risk loss and the bias for each client;    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | both are essential for any ML model training, whether centralized or            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | decentralized. We formulate EFFL as a constrained multi-constrained             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | multi-objectives optimization (MCMOO) problem, with the decision bias and       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | egalitarian fairness as constraints and the minimization of the empirical risk  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | losses on all clients as multiple objectives to be optimized. We propose a      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | gradient-based three-stage algorithm to obtain the Pareto optimal solutions     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | within the constraint space. Extensive experiments demonstrate that EFFL        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | outperforms other state-of-the-art FL algorithms in achieving a                 |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | high-performance global model with enhanced egalitarian fairness among all      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | clients.                                                                        |                                   |                           |
| End-to-end Risk Prediction of Atrial Fibrillation from the 12-Lead ECG by Deep Neural Networks                                               | Theogene Habineza, Antônio H. Ribeiro, Daniel Gedon, Joachim A. Behar, Antonio Luiz P. Ribeiro, Thomas B. Schön                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Background: Atrial fibrillation (AF) is one of the most common cardiac          | http://arxiv.org/abs/2309.16335v1 | 2023-09-28 10:47:40+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | arrhythmias that affects millions of people each year worldwide and it is       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | closely linked to increased risk of cardiovascular diseases such as stroke and  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | heart failure. Machine learning methods have shown promising results in         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | evaluating the risk of developing atrial fibrillation from the                  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | electrocardiogram. We aim to develop and evaluate one such algorithm on a large |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | CODE dataset collected in Brazil.                                               |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |   Results: The deep neural network model identified patients without indication |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of AF in the presented ECG but who will develop AF in the future with an AUC    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | score of 0.845. From our survival model, we obtain that patients in the         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | high-risk group (i.e. with the probability of a future AF case being greater    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | than 0.7) are 50% more likely to develop AF within 40 weeks, while patients     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | belonging to the minimal-risk group (i.e. with the probability of a future AF   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | case being less than or equal to 0.1) have more than 85% chance of remaining AF |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | free up until after seven years.                                                |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |   Conclusion: We developed and validated a model for AF risk prediction. If     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applied in clinical practice, the model possesses the potential of providing    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | valuable and useful information in decision-making and patient management       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | processes.                                                                      |                                   |                           |
| CloudProphet: A Machine Learning-Based Performance Prediction for Public Clouds                                                              | Darong Huang, Luis Costero, Ali Pahlevan, Marina Zapater, David Atienza                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | Computing servers have played a key role in developing and processing           | http://arxiv.org/abs/2309.16333v1 | 2023-09-28 10:45:11+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | emerging compute-intensive applications in recent years. Consolidating multiple |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | virtual machines (VMs) inside one server to run various applications introduces |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | severe competence for limited resources among VMs. Many techniques such as VM   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | scheduling and resource provisioning are proposed to maximize the               |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | cost-efficiency of the computing servers while alleviating the performance      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | inference between VMs. However, these management techniques require accurate    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance prediction of the application running inside the VM, which is       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | challenging to get in the public cloud due to the black-box nature of the VMs.  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | From this perspective, this paper proposes a novel machine learning-based       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance prediction approach for applications running in the cloud. To       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | achieve high accuracy predictions for black-box VMs, the proposed method first  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | identifies the running application inside the virtual machine. It then selects  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | highly-correlated runtime metrics as the input of the machine learning approach |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to accurately predict the performance level of the cloud application.           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Experimental results with state-of-the-art cloud benchmarks demonstrate that    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | our proposed method outperforms the existing prediction methods by more than 2x |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in terms of worst prediction error. In addition, we successfully tackle the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | challenge in performance prediction for applications with variable workloads by |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | introducing the performance degradation index, which other comparison methods   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fail to consider. The workflow versatility of the proposed approach has been    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | verified with different modern servers and VM configurations.                   |                                   |                           |
| Valley-polarization and stable triplet exciton formation in 2D lateral heterostrcuture of hBN-kagome and graphene                            | Souren Adhikary, Sasmita Mohakud, Sudipta Dutta                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Broken spatial inversion symmetry in semiconducting materials with              | http://arxiv.org/abs/2309.16325v1 | 2023-09-28 10:35:16+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | time-reversal pair valleys can exhibit valley polarization. Based on            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | first-principles calculations, here we propose a lateral heterostructure of     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | kagome lattice of hBN and hexagonal graphene domains that exhibits opposite     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Berry curvature in inequivalent K and K' valleys. Explicit consideration of     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | excitonic scattering processes within GW and Bethe-Salpeter equation formalism  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | confirm insignificant intervalley coupling and consequent valley polarization   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ability along with 0.46 eV higher binding energy of triplet excitons. Such      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | heterostructure with large charge carrier mobility can be exploited for         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | advanced valleytronic and optoelectronic applications.                          |                                   |                           |
| Locating clustered seismicity using Distance Geometry Solvers: applications for sparse and single-borehole DAS networks                      | Katinka Tuinstra, Francesco Grigoli, Federica Lanza, Antonio Pio Rinaldi, Andreas Fichtner, Stefan Wiemer                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | The determination of seismic event locations with sparse networks or            | http://arxiv.org/abs/2309.16317v1 | 2023-09-28 10:13:34+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | single-borehole systems remains a significant challenge in observational        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | seismology. Leveraging the advantages of the location approach HADES, which was |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | initially developed for locating clustered seismicity recorded at two stations, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | we present here an improved version of the methodology: HADES-R. Where HADES    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | previously needed a minimum of 4 absolutely located master events, HADES-R      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | solves a least-squares problem to find the relative inter-event distances in    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the cluster, and uses only a single master event to find the locations of all   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | events, and subsequently applies rotational optimiser to find the cluster       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | orientation. It can leverage iterative station combinations if multiple         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | receivers are available, to describe the cluster shape and orientation          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | uncertainty with a bootstrap approach. The improved method requires P- and      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | S-phase arrival picks, a homogeneous velocity model, a single master event with |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | a known location, and an estimate of the cluster width. The approach is         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | benchmarked on the 2019 Ridgecrest sequence recorded at two stations, and       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applied to two seismic clusters at the FORGE geothermal test site, including a  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | microseismic monitoring scenario with a DAS in a vertical borehole. Traditional |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | procedures struggle in these settings due to the ill-posed network              |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | configuration. The azimuthal ambiguity in this scenario is partially overcome   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | by assuming that all events belong to the same cluster around the master event  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and a cluster width estimate. We find the cluster shape in both cases, although |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the orientation remains uncertain. The method's ability to constrain the        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | cluster shape and location with only one well-located event offers promising    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | implications, especially for environments where limited or specialised          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | instrumentation is in use.                                                      |                                   |                           |
| Astroconformer: The Prospects of Analyzing Stellar Light Curves with Transformer-Based Deep Learning Models                                  | Jia-Shu Pan, Yuan-Sen Ting, Jie Yu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Light curves of stars encapsulate a wealth of information about stellar         | http://arxiv.org/abs/2309.16316v1 | 2023-09-28 10:13:23+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | oscillations and granulation, thereby offering key insights into the internal   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | structure and evolutionary state of stars. Conventional asteroseismic           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | techniques have been largely confined to power spectral analysis, neglecting    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the valuable phase information contained within light curves. While recent      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | machine learning applications in asteroseismology utilizing Convolutional       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Neural Networks (CNNs) have successfully inferred stellar attributes from light |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | curves, they are often limited by the local feature extraction inherent in      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | convolutional operations. To circumvent these constraints, we present           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | $\textit{Astroconformer}$, a Transformer-based deep learning framework designed |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to capture long-range dependencies in stellar light curves. Our empirical       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | analysis, which focuses on estimating surface gravity ($\log g$), is grounded   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in a carefully curated dataset derived from $\textit{Kepler}$ light curves.     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | These light curves feature asteroseismic $\log g$ values spanning from 0.2 to   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 4.4. Our results underscore that, in the regime where the training data is      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | abundant, $\textit{Astroconformer}$ attains a root-mean-square-error (RMSE) of  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.017 dex around $\log g \approx 3 $. Even in regions where training data are   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | sparse, the RMSE can reach 0.1 dex. It outperforms not only the K-nearest       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | neighbor-based model ($\textit{The SWAN}$) but also state-of-the-art CNNs.      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Ablation studies confirm that the efficacy of the models in this particular     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | task is strongly influenced by the size of their receptive fields, with larger  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | receptive fields correlating with enhanced performance. Moreover, we find that  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the attention mechanisms within $\textit{Astroconformer}$ are well-aligned with |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the inherent characteristics of stellar oscillations and granulation present in |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the light curves.                                                               |                                   |                           |
| A Primer on Bayesian Neural Networks: Review and Debates                                                                                     | Julyan Arbel, Konstantinos Pitas, Mariia Vladimirova, Vincent Fortuin                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Neural networks have achieved remarkable performance across various problem     | http://arxiv.org/abs/2309.16314v1 | 2023-09-28 10:09:15+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | domains, but their widespread applicability is hindered by inherent limitations |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | such as overconfidence in predictions, lack of interpretability, and            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | vulnerability to adversarial attacks. To address these challenges, Bayesian     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | neural networks (BNNs) have emerged as a compelling extension of conventional   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | neural networks, integrating uncertainty estimation into their predictive       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | capabilities.                                                                   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |   This comprehensive primer presents a systematic introduction to the           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fundamental concepts of neural networks and Bayesian inference, elucidating     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | their synergistic integration for the development of BNNs. The target audience  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | comprises statisticians with a potential background in Bayesian methods but     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | lacking deep learning expertise, as well as machine learners proficient in deep |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | neural networks but with limited exposure to Bayesian statistics. We provide an |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | overview of commonly employed priors, examining their impact on model behavior  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and performance. Additionally, we delve into the practical considerations       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | associated with training and inference in BNNs.                                 |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |   Furthermore, we explore advanced topics within the realm of BNN research,     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | acknowledging the existence of ongoing debates and controversies. By offering   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | insights into cutting-edge developments, this primer not only equips            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | researchers and practitioners with a solid foundation in BNNs, but also         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | illuminates the potential applications of this dynamic field. As a valuable     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | resource, it fosters an understanding of BNNs and their promising prospects,    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | facilitating further advancements in the pursuit of knowledge and innovation.   |                                   |                           |
| Audio Visual Speaker Localization from EgoCentric Views                                                                                      | Jinzheng Zhao, Yong Xu, Xinyuan Qian, Wenwu Wang                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | The use of audio and visual modality for speaker localization has been well     | http://arxiv.org/abs/2309.16308v1 | 2023-09-28 10:01:08+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | studied in the literature by exploiting their complementary characteristics.    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | However, most previous works employ the setting of static sensors mounted at    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fixed positions. Unlike them, in this work, we explore the ego-centric setting, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | where the heterogeneous sensors are embodied and could be moving with a human   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to facilitate speaker localization. Compared to the static scenario, the        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ego-centric setting is more realistic for smart-home applications e.g., a       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | service robot. However, this also brings new challenges such as blurred images, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | frequent speaker disappearance from the field of view of the wearer, and        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | occlusions. In this paper, we study egocentric audio-visual speaker DOA         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | estimation and deal with the challenges mentioned above. Specifically, we       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | propose a transformer-based audio-visual fusion method to estimate the relative |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | DOA of the speaker to the wearer, and design a training strategy to mitigate    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the problem of the speaker disappearing from the camera's view. We also develop |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | a new dataset for simulating the out-of-view scenarios, by creating a scene     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | with a camera wearer walking around while a speaker is moving at the same time. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | The experimental results show that our proposed method offers promising         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance in this new dataset in terms of tracking accuracy. Finally, we      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | adapt the proposed method for the multi-speaker scenario. Experiments on        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | EasyCom show the effectiveness of the proposed model for multiple speakers in   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | real scenarios, which achieves state-of-the-art results in the sphere active    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | speaker detection task and the wearer activity prediction task. The simulated   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | dataset and related code are available at                                       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | https://github.com/KawhiZhao/Egocentric-Audio-Visual-Speaker-Localization.      |                                   |                           |
| Forming complex neurons by four-wave mixing in a Bose-Einstein condensate                                                                    | Kai Niklas Hansmann, Reinhold Walser                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | A physical artificial complex-valued neuron is formed by four-wave mixing in    | http://arxiv.org/abs/2309.16302v1 | 2023-09-28 09:54:28+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | a homogeneous three-dimensional Bose-Einstein condensate. Bragg beamsplitter    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pulses prepare superpositions of three plane-waves states as an input- and the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fourth wave as an output signal. The nonlinear dynamics of the non-degenerate   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | four-wave mixing process leads to Josephson-like oscillations within the closed |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | four-dimensional subspace and defines the activation function of a neuron. Due  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to the high number of symmetries, closed form solutions can be found by         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quadrature and agree with numerical simulation. The ideal behaviour of an       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | isolated four-wave mixing setup is compared to a situation with additional      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | population of rogue states. We observe a robust persistence of the main         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | oscillation. As an application for neural learning of this physical system, we  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | train it on the XOR problem. After $100$ training epochs, the neuron responds   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to input data correctly at the $10^{-5}$ error level.                           |                                   |                           |
| At Which Training Stage Does Cocde Data Help LLMs Reasoning?                                                                                 | Yingwei Ma, Yue Liu, Yue Yu, Yuanliang Zhang, Yu Jiang, Changjian Wang, Shanshan Li                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Large Language Models (LLMs) have exhibited remarkable reasoning capabilities   | http://arxiv.org/abs/2309.16298v1 | 2023-09-28 09:50:27+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and become the foundation of language technologies. Inspired by the great       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | success of code data in training LLMs, we naturally wonder at which training    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | stage introducing code data can really help LLMs reasoning. To this end, this   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | paper systematically explores the impact of code data on LLMs at different      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | stages. Concretely, we introduce the code data at the pre-training stage,       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | instruction-tuning stage, and both of them, respectively. Then, the reasoning   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | capability of LLMs is comprehensively and fairly evaluated via six reasoning    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | tasks in five domains. We critically analyze the experimental results and       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | provide conclusions with insights. First, pre-training LLMs with the mixture of |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | code and text can significantly enhance LLMs' general reasoning capability      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | almost without negative transfer on other tasks. Besides, at the                |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | instruction-tuning stage, code data endows LLMs the task-specific reasoning     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | capability. Moreover, the dynamic mixing strategy of code and text data assists |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | LLMs to learn reasoning capability step-by-step during training. These insights |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | deepen the understanding of LLMs regarding reasoning ability for their          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | application, such as scientific question answering, legal support, etc. The     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | source code and model parameters are released at the                            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | link:~\url{https://github.com/yingweima2022/CodeLLM}.                           |                                   |                           |
| LawBench: Benchmarking Legal Knowledge of Large Language Models                                                                              | Zhiwei Fei, Xiaoyu Shen, Dawei Zhu, Fengzhe Zhou, Zhuo Han, Songyang Zhang, Kai Chen, Zongwen Shen, Jidong Ge                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | Large language models (LLMs) have demonstrated strong capabilities in various   | http://arxiv.org/abs/2309.16289v1 | 2023-09-28 09:35:59+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | aspects. However, when applying them to the highly specialized, safe-critical   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | legal domain, it is unclear how much legal knowledge they possess and whether   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | they can reliably perform legal-related tasks. To address this gap, we propose  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | a comprehensive evaluation benchmark LawBench. LawBench has been meticulously   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | crafted to have precise assessment of the LLMs' legal capabilities from three   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | cognitive levels: (1) Legal knowledge memorization: whether LLMs can memorize   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | needed legal concepts, articles and facts; (2) Legal knowledge understanding:   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | whether LLMs can comprehend entities, events and relationships within legal     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | text; (3) Legal knowledge applying: whether LLMs can properly utilize their     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | legal knowledge and make necessary reasoning steps to solve realistic legal     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | tasks. LawBench contains 20 diverse tasks covering 5 task types: single-label   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | classification (SLC), multi-label classification (MLC), regression, extraction  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and generation. We perform extensive evaluations of 51 LLMs on LawBench,        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | including 20 multilingual LLMs, 22 Chinese-oriented LLMs and 9 legal specific   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | LLMs. The results show that GPT-4 remains the best-performing LLM in the legal  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | domain, surpassing the others by a significant margin. While fine-tuning LLMs   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | on legal specific text brings certain improvements, we are still a long way     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | from obtaining usable and reliable LLMs in legal tasks. All data, model         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | predictions and evaluation code are released in                                 |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | https://github.com/open-compass/LawBench/. We hope this benchmark provides      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in-depth understanding of the LLMs' domain-specified capabilities and speed up  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the development of LLMs in the legal domain.                                    |                                   |                           |
| Generalizable Heterogeneous Federated Cross-Correlation and Instance Similarity Learning                                                     | Wenke Huang, Mang Ye, Zekun Shi, Bo Du                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Federated learning is an important privacy-preserving multi-party learning      | http://arxiv.org/abs/2309.16286v1 | 2023-09-28 09:32:27+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | paradigm, involving collaborative learning with others and local updating on    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | private data. Model heterogeneity and catastrophic forgetting are two crucial   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | challenges, which greatly limit the applicability and generalizability. This    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | paper presents a novel FCCL+, federated correlation and similarity learning     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | with non-target distillation, facilitating the both intra-domain                |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | discriminability and inter-domain generalization. For heterogeneity issue, we   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | leverage irrelevant unlabeled public data for communication between the         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | heterogeneous participants. We construct cross-correlation matrix and align     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | instance similarity distribution on both logits and feature levels, which       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | effectively overcomes the communication barrier and improves the generalizable  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ability. For catastrophic forgetting in local updating stage, FCCL+ introduces  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Federated Non Target Distillation, which retains inter-domain knowledge while   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | avoiding the optimization conflict issue, fulling distilling privileged         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | inter-domain information through depicting posterior classes relation.          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Considering that there is no standard benchmark for evaluating existing         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | heterogeneous federated learning under the same setting, we present a           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | comprehensive benchmark with extensive representative methods under four domain |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | shift scenarios, supporting both heterogeneous and homogeneous federated        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | settings. Empirical results demonstrate the superiority of our method and the   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | efficiency of modules on various scenarios.                                     |                                   |                           |
| Reverse Hölder inequalities on the space of Kähler metrics of a Fano variety and log-concavity                                               | Robert J. Berman                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | A reverse Holder inequality is established on the space of Kahler metrics in    | http://arxiv.org/abs/2309.16278v1 | 2023-09-28 09:20:02+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the first Chern class of a Fano manifold X endowed with Darvas Finsler metrics. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | The inequality holds under a uniform bound on a twisted Ricci potential and     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | extends to the general setup of Fano varieties. Its proof leverages a "hidden"  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | log-concavity. An application to destabilizing geodesic rays is provided, which |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | yields a reverse Holder inequality for the speed of the geodesic. In the case   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of Aubin's continuity path the constant in the corresponding Holder bound is    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | shown to be universal when X is K-unstable. Some intruiging relations to        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Harnack bounds and the partial C^0-estimate are pointed out.                    |                                   |                           |
| Light-activated ferroelectric transition in layer dependent Bi2O2Se films                                                                    | Hao Li, Adeela Nairan, Xiaoran Niu, Yuxiang Chen, Huarui Sun, Linqing Lai, Jingkai Qin, Leyang Dang, Guigen Wang, Usman Khan, Feng He                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Bi2O2Se has attracted intensive attention due to its potential in               | http://arxiv.org/abs/2309.16276v1 | 2023-09-28 09:19:23+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | electronics, optoelectronics, as well as ferroelectric applications. Despite    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | that, there have only been a handful of experimental studies based on ultrafast |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | spectroscopy to elucidate the carrier dynamics in Bi2O2Se thin films, Different |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | groups have reported various ultrafast timescales and associated mechanisms     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | across films of different thicknesses. A comprehensive understanding in         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | relation to thickness and fluence is still lacking. In this work, we have       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | systematically explored the thickness-dependent Raman spectroscopy and          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ultrafast carrier dynamics in chemical vapor deposition (CVD)-grown Bi2O2Se     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | thin films on mica substrate with thicknesses varying from 22.44 nm down to     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 4.62 nm at both low and high pump fluence regions. Combining the thickness      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | dependence and fluence dependence of the slow decay time, we demonstrate a      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ferroelectric transition in the thinner (< 8 nm) Bi2O2Se films, influenced by   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | substrate-induced compressive strain and non-equilibrium states. Moreover, this |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | transition can be manifested under highly non-equilibrium states. Our results   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | deepen the understanding of the interplay between the ferroelectric phase and   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | semiconducting characteristics of Bi2O2Se thin films, providing a new route to  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | manipulate the ferroelectric transition.                                        |                                   |                           |
| Hyper-reduction for Petrov-Galerkin reduced order models                                                                                     | S. Ares de Parga, J. R. Bravo, J. A. Hernandez, R. Zorrilla, R. Rossi                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Projection-based Reduced Order Models minimize the discrete residual of a       | http://arxiv.org/abs/2309.16267v1 | 2023-09-28 09:03:31+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | "full order model" (FOM) while constraining the unknowns to a reduced dimension |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | space. For problems with symmetric positive definite (SPD) Jacobians, this is   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | optimally achieved by projecting the full order residual onto the approximation |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | basis (Galerkin Projection). This is sub-optimal for non-SPD Jacobians as it    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | only minimizes the projection of the residual, not the residual itself. An      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | alternative is to directly minimize the 2-norm of the residual, achievable      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | using QR factorization or the method of the normal equations (LSPG). The first  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | approach involves constructing and factorizing a large matrix, while LSPG       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | avoids this but requires constructing a product element by element,             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | necessitating a complementary mesh and adding complexity to the hyper-reduction |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | process. This work proposes an alternative based on Petrov-Galerkin             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | minimization. We choose a left basis for a least-squares minimization on a      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | reduced problem, ensuring the discrete full order residual is minimized. This   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | is applicable to both SPD and non-SPD Jacobians, allowing element-by-element    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | assembly, avoiding the use of a complementary mesh, and simplifying finite      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | element implementation. The technique is suitable for hyper-reduction using the |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Empirical Cubature Method and is applicable in nonlinear reduction procedures.  |                                   |                           |
| GAMMA: Generalizable Articulation Modeling and Manipulation for Articulated Objects                                                          | Qiaojun Yu, Junbo Wang, Wenhai Liu, Ce Hao, Liu Liu, Lin Shao, Weiming Wang, Cewu Lu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | Articulated objects like cabinets and doors are widespread in daily life.       | http://arxiv.org/abs/2309.16264v1 | 2023-09-28 08:57:14+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | However, directly manipulating 3D articulated objects is challenging because    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | they have diverse geometrical shapes, semantic categories, and kinetic          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | constraints. Prior works mostly focused on recognizing and manipulating         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | articulated objects with specific joint types. They can either estimate the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | joint parameters or distinguish suitable grasp poses to facilitate trajectory   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | planning. Although these approaches have succeeded in certain types of          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | articulated objects, they lack generalizability to unseen objects, which        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | significantly impedes their application in broader scenarios. In this paper, we |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | propose a novel framework of Generalizable Articulation Modeling and            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Manipulating for Articulated Objects (GAMMA), which learns both articulation    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | modeling and grasp pose affordance from diverse articulated objects with        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | different categories. In addition, GAMMA adopts adaptive manipulation to        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | iteratively reduce the modeling errors and enhance manipulation performance. We |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | train GAMMA with the PartNet-Mobility dataset and evaluate with comprehensive   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | experiments in SAPIEN simulation and real-world Franka robot arms. Results show |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | that GAMMA significantly outperforms SOTA articulation modeling and             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | manipulation algorithms in unseen and cross-category articulated objects. We    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | will open-source all codes and datasets in both simulation and real robots for  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | reproduction in the final version. Images and videos are published on the       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | project website at: http://sites.google.com/view/gamma-articulation             |                                   |                           |
| Cooperation Dynamics in Multi-Agent Systems: Exploring Game-Theoretic Scenarios with Mean-Field Equilibria                                   | Vaigarai Sathi, Sabahat Shaik, Jaswanth Nidamanuri                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Cooperation is fundamental in Multi-Agent Systems (MAS) and Multi-Agent         | http://arxiv.org/abs/2309.16263v1 | 2023-09-28 08:57:01+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Reinforcement Learning (MARL), often requiring agents to balance individual     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | gains with collective rewards. In this regard, this paper aims to investigate   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | strategies to invoke cooperation in game-theoretic scenarios, namely the        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Iterated Prisoner's Dilemma, where agents must optimize both individual and     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | group outcomes. Existing cooperative strategies are analyzed for their          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | effectiveness in promoting group-oriented behavior in repeated games.           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Modifications are proposed where encouraging group rewards will also result in  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | a higher individual gain, addressing real-world dilemmas seen in distributed    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | systems. The study extends to scenarios with exponentially growing agent        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | populations ($N \longrightarrow +\infty$), where traditional computation and    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | equilibrium determination are challenging. Leveraging mean-field game theory,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | equilibrium solutions and reward structures are established for infinitely      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | large agent sets in repeated games. Finally, practical insights are offered     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | through simulations using the Multi Agent-Posthumous Credit Assignment trainer, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and the paper explores adapting simulation algorithms to create scenarios       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | favoring cooperation for group rewards. These practical implementations bridge  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | theoretical concepts with real-world applications.                              |                                   |                           |
| Dilation theorem via Schrödingerisation, with applications to the quantum simulation of differential equations                               | Junpeng Hu, Shi Jin, Nana Liu, Lei Zhang                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Nagy's unitary dilation theorem in operator theory asserts the possibility of   | http://arxiv.org/abs/2309.16262v1 | 2023-09-28 08:55:43+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | dilating a contraction into a unitary operator. When used in quantum computing, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | its practical implementation primarily relies on block-encoding techniques,     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | based on finite-dimensional scenarios. In this study, we delve into the         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | recently devised Schr\"odingerisation approach and demonstrate its viability as |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | an alternative dilation technique. This approach is applicable to operators in  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the form of $V(t)=\exp(-At)$, which arises in wide-ranging applications,        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | particularly in solving linear ordinary and partial differential equations.     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Importantly, the Schr\"odingerisation approach is adaptable to both finite and  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | infinite-dimensional cases, in both countable and uncountable domains. For      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quantum systems lying in infinite dimensional Hilbert space, the dilation       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | involves adding a single infinite dimensional mode, and this is the             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | continuous-variable version of the Schr\"odingerisation procedure which makes   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | it suitable for analog quantum computing. Furthermore, by discretising          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | continuous variables, the Schr\"odingerisation method can also be effectively   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | employed in finite-dimensional scenarios suitable for qubit-based quantum       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | computing.                                                                      |                                   |                           |
| QonFusion -- Quantum Approaches to Gaussian Random Variables: Applications in Stable Diffusion and Brownian Motion                           | Shlomo Kashani                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | In the present study, we delineate a strategy focused on non-parametric         | http://arxiv.org/abs/2309.16258v1 | 2023-09-28 08:51:18+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quantum circuits for the generation of Gaussian random variables (GRVs). This   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quantum-centric approach serves as a substitute for conventional pseudorandom   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | number generators (PRNGs), such as the \textbf{torch.rand} function in PyTorch. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | The principal theme of our research is the incorporation of Quantum Random      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Number Generators (QRNGs) into classical models of diffusion. Notably, our      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Quantum Gaussian Random Variable Generator fulfills dual roles, facilitating    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | simulations in both Stable Diffusion (SD) and Brownian Motion (BM). This        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | diverges markedly from prevailing methods that utilize parametric quantum       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | circuits (PQCs), often in conjunction with variational quantum eigensolvers     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (VQEs). Although conventional techniques can accurately approximate ground      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | states in complex systems or model elaborate probability distributions, they    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | require a computationally demanding optimization process to tune parameters.    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Our non-parametric strategy obviates this necessity. To facilitate assimilating |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | our methodology into existing computational frameworks, we put forward          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | QonFusion, a Python library congruent with both PyTorch and PennyLane,          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | functioning as a bridge between classical and quantum computational paradigms.  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | We validate QonFusion through extensive statistical testing, including tests    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | which confirm the statistical equivalence of the Gaussian samples from our      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quantum approach to classical counterparts within defined significance limits.  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | QonFusion is available at                                                       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | \url{https://boltzmannentropy.github.io/qonfusion.github.io/} to reproduce all  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | findings here.                                                                  |                                   |                           |
| Nondestructive chicken egg fertility detection using CNN-transfer learning algorithms                                                        | Shoffan Saifullah, Rafal Drezewski, Anton Yudhana, Andri Pranolo, Wilis Kaswijanti, Andiko Putro Suryotomo, Seno Aji Putra, Alin Khaliduzzaman, Anton Satria Prabuwono, Nathalie Japkowicz                                                                                                                                                                                                                                                                                                                                                                                                  | This study explored the application of CNN-Transfer Learning for                | http://arxiv.org/abs/2309.16257v1 | 2023-09-28 08:50:19+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | nondestructive chicken egg fertility detection for precision poultry hatchery   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | practices. Four models, VGG16, ResNet50, InceptionNet, and MobileNet, were      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | trained and evaluated on a dataset (200 single egg images) using augmented      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | images (rotation, flip, scale, translation, and reflection). Although the       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | training results demonstrated that all models achieved high accuracy,           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | indicating their ability to accurately learn and classify chicken eggs'         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fertility state, when evaluated on the testing set, variations in accuracy and  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance were observed. InceptionNet exhibited the best overall performance, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | accurately classifying fertile and non-fertile eggs. It demonstrated excellent  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance in both training and testing sets in all parameters of the          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | evaluation metrics. In testing set, it achieved an accuracy of 0.98, a          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | sensitivity of 1 for detecting fertile eggs, and a specificity of 0.96 for      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | identifying non-fertile eggs. The higher performance is attributed to its       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | unique architecture efficiently capturing features at different scales leading  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to improved accuracy and robustness. Further optimization and fine-tuning of    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the models might necessary to address the limitations in accurately detecting   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fertile and non-fertile eggs in case of other models. This study highlighted    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the potential of CNN-Transfer Learning for nondestructive fertility detection   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and emphasizes the need for further research to enhance the models'             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | capabilities and ensure accurate classification.                                |                                   |                           |
| Spider4SPARQL: A Complex Benchmark for Evaluating Knowledge Graph Question Answering Systems                                                 | Catherine Kosten, Philippe Cudré-Mauroux, Kurt Stockinger                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | With the recent spike in the number and availability of Large Language Models   | http://arxiv.org/abs/2309.16248v1 | 2023-09-28 08:41:08+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (LLMs), it has become increasingly important to provide large and realistic     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | benchmarks for evaluating Knowledge Graph Question Answering (KBQA) systems. So |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | far the majority of benchmarks rely on pattern-based SPARQL query generation    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | approaches. The subsequent natural language (NL) question generation is         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | conducted through crowdsourcing or other automated methods, such as rule-based  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | paraphrasing or NL question templates. Although some of these datasets are of   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | considerable size, their pitfall lies in their pattern-based generation         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | approaches, which do not always generalize well to the vague and linguistically |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | diverse questions asked by humans in real-world contexts.                       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |   In this paper, we introduce Spider4SPARQL - a new SPARQL benchmark dataset    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | featuring 9,693 previously existing manually generated NL questions and 4,721   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | unique, novel, and complex SPARQL queries of varying complexity. In addition to |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the NL/SPARQL pairs, we also provide their corresponding 166 knowledge graphs   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and ontologies, which cover 138 different domains. Our complex benchmark        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | enables novel ways of evaluating the strengths and weaknesses of modern KGQA    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | systems. We evaluate the system with state-of-the-art KGQA systems as well as   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | LLMs, which achieve only up to 45\% execution accuracy, demonstrating that      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Spider4SPARQL is a challenging benchmark for future research.                   |                                   |                           |
| PP-MeT: a Real-world Personalized Prompt based Meeting Transcription System                                                                  | Xiang Lyu, Yuhang Cao, Qing Wang, Jingjing Yin, Yuguang Yang, Pengpeng Zou, Yanni Hu, Heng Lu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | Speaker-attributed automatic speech recognition (SA-ASR) improves the           | http://arxiv.org/abs/2309.16247v1 | 2023-09-28 08:40:48+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | accuracy and applicability of multi-speaker ASR systems in real-world scenarios |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | by assigning speaker labels to transcribed texts. However, SA-ASR poses unique  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | challenges due to factors such as speaker overlap, speaker variability,         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | background noise, and reverberation. In this study, we propose PP-MeT system, a |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | real-world personalized prompt based meeting transcription system, which        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | consists of a clustering system, target-speaker voice activity detection        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (TS-VAD), and TS-ASR. Specifically, we utilize target-speaker embedding as a    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | prompt in TS-VAD and TS-ASR modules in our proposed system. In constrast with   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | previous system, we fully leverage pre-trained models for system                |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | initialization, thereby bestowing our approach with heightened generalizability |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and precision. Experiments on M2MeT2.0 Challenge dataset show that our system   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | achieves a cp-CER of 11.27% on the test set, ranking first in both fixed and    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | open training conditions.                                                       |                                   |                           |
| The Hyper Suprime-Cam extended Point Spread Functions and applications to measuring the intra-halo light                                     | L. P. Garate-Nuñez, A. S. G. Robotham, S. Bellstedt, L. J. M. Davies, C. Martínez-Lombilla                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | We present extended point spread function (PSF) models for the Hyper            | http://arxiv.org/abs/2309.16244v1 | 2023-09-28 08:35:52+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Suprime-Cam Subaru Strategic Program Public Data Release 3 (HSC-SSP PDR3) in    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | all $\textit{g,r,i,Z}$ and $\textit{Y}$-bands. Due to its 8.2m primary mirror   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and long exposure periods, HSC combines deep images with wide-field coverage,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | making it one of the most suitable observing facilities for low surface         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | brightness (LSB) studies. By applying a median stacking technique of point      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | sources with different brightnesses, we show how to construct the HSC-SSP PDR3  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | PSF models to an extent of R $\sim$ 5.6 arcmin. These new PSFs provide the      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | community with a crucial tool to characterise LSB properties at large angles.   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | We apply our HSC PSFs and demonstrate that they behave reasonably in two cases: |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | first, to generate a 2-D model of a bright star, and second, to remove the      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | PSF-scattered light from an Ultra Deep image of the 400020 Galaxy And Mass      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Assembly (GAMA) group in the SXDS field. Our main focus in this second          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | application is characterising the $\textit{r}$-band intra-halo light (IHL)      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | component of 400020. Building on advanced source extraction techniques with     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | careful consideration of PSF flux, we measure the IHL surface brightness (SB)   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | group profile up to $\sim$ 31 mag arcsec$^{-2}$ and R = 300 kpc. We estimate    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the IHL fraction ($\mathrm{f_{IHL}}$) profile, with a mean of                   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | $\mathrm{f_{IHL}}$ $\sim$ 0.13. Our results show that not removing the PSF      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | light can overestimate the IHL SB by $\sim$ 1.7 mag arcsec$^{-2}$ and the       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | $\mathrm{f_{IHL}}$ by $\sim$ 30%.                                               |                                   |                           |
| Limitations of local update recovery in stabilizer-GKP codes: a quantum optimal transport approach                                           | Robert König, Cambyse Rouzé                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | Local update recovery seeks to maintain quantum information by applying local   | http://arxiv.org/abs/2309.16241v1 | 2023-09-28 08:29:52+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | correction maps alternating with and compensating for the action of noise.      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Motivated by recent constructions based on quantum LDPC codes in the            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | finite-dimensional setting, we establish an analytic upper bound on the         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fault-tolerance threshold for concatenated GKP-stabilizer codes with local      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | update recovery. Our bound applies to noise channels that are tensor products   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of one-mode beamsplitters with arbitrary environment states, capturing, in      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | particular, photon loss occurring independently in each mode. It shows that for |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | loss rates above a threshold given explicitly as a function of the locality of  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the recovery maps, encoded information is lost at an exponential rate. This     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | extends an early result by Razborov from discrete to continuous variable (CV)   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quantum systems.                                                                |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |   To prove our result, we study a metric on bosonic states akin to the          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Wasserstein distance between two CV density functions, which we call the        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | bosonic Wasserstein distance. It can be thought of as a CV extension of a       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | quantum Wasserstein distance of order 1 recently introduced by De Palma et al.  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in the context of qudit systems, in the sense that it captures the notion of    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | locality in a CV setting. We establish several basic properties, including a    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | relation to the trace distance and diameter bounds for states with finite       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | average photon number. We then study its contraction properties under quantum   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | channels, including tensorization, locality and strict contraction under        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | beamsplitter-type noise channels. Due to the simplicity of its formulation, and |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the established wide applicability of its finite-dimensional counterpart, we    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | believe that the bosonic Wasserstein distance will become a versatile tool in   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the study of CV quantum systems.                                                |                                   |                           |
| Beyond Reverse KL: Generalizing Direct Preference Optimization with Diverse Divergence Constraints                                           | Chaoqi Wang, Yibo Jiang, Chenghao Yang, Han Liu, Yuxin Chen                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | The increasing capabilities of large language models (LLMs) raise               | http://arxiv.org/abs/2309.16240v1 | 2023-09-28 08:29:44+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | opportunities for artificial general intelligence but concurrently amplify      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | safety concerns, such as potential misuse of AI systems, necessitating          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | effective AI alignment. Reinforcement Learning from Human Feedback (RLHF) has   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | emerged as a promising pathway towards AI alignment but brings forth challenges |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | due to its complexity and dependence on a separate reward model. Direct         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Preference Optimization (DPO) has been proposed as an alternative, and it       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | remains equivalent to RLHF under the reverse KL regularization constraint. This |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | paper presents $f$-DPO, a generalized approach to DPO by incorporating diverse  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | divergence constraints. We show that under certain $f$-divergences, including   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Jensen-Shannon divergence, forward KL divergences and $\alpha$-divergences, the |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | complex relationship between the reward and optimal policy can also be          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | simplified by addressing the Karush-Kuhn-Tucker conditions. This eliminates the |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | need for estimating the normalizing constant in the Bradley-Terry model and     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | enables a tractable mapping between the reward function and the optimal policy. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Our approach optimizes LLMs to align with human preferences in a more efficient |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and supervised manner under a broad set of divergence constraints. Empirically, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | adopting these divergences ensures a balance between alignment performance and  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | generation diversity. Importantly, $f$-DPO outperforms PPO-based methods in     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | divergence efficiency, and divergence constraints directly influence expected   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | calibration error (ECE).                                                        |                                   |                           |
| Human spatial dynamics for electricity demand forecasting: the case of France during the 2022 energy crisis                                  | Nathan Doumèche, Yann Allioux, Yannig Goude, Stefania Rubrichi                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | Accurate electricity demand forecasting is crucial to meet energy security      | http://arxiv.org/abs/2309.16238v1 | 2023-09-28 08:23:26+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and efficiency, especially when relying on intermittent renewable energy        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | sources. Recently, massive savings have been observed in Europe, following an   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | unprecedented global energy crisis. However, assessing the impact of such       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | crisis and of government incentives on electricity consumption behaviour is     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | challenging. Moreover, standard statistical models based on meteorological and  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | calendar data have difficulty adapting to such brutal changes. Here, we show    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | that mobility indices based on mobile network data significantly improve the    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance of the state-of-the-art models in electricity demand forecasting    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | during the sobriety period. We start by documenting the drop in the French      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | electricity consumption during the winter of 2022-2023. We then show how our    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | mobile network data captures work dynamics and how adding these mobility        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | indices outperforms the state-of-the-art during this atypical period. Our       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | results characterise the effect of work behaviours on the electricity demand.   |                                   |                           |
| Object Motion Guided Human Motion Synthesis                                                                                                  | Jiaman Li, Jiajun Wu, C. Karen Liu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Modeling human behaviors in contextual environments has a wide range of         | http://arxiv.org/abs/2309.16237v1 | 2023-09-28 08:22:00+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications in character animation, embodied AI, VR/AR, and robotics. In       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | real-world scenarios, humans frequently interact with the environment and       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | manipulate various objects to complete daily tasks. In this work, we study the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problem of full-body human motion synthesis for the manipulation of large-sized |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | objects. We propose Object MOtion guided human MOtion synthesis (OMOMO), a      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | conditional diffusion framework that can generate full-body manipulation        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | behaviors from only the object motion. Since naively applying diffusion models  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fails to precisely enforce contact constraints between the hands and the        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | object, OMOMO learns two separate denoising processes to first predict hand     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | positions from object motion and subsequently synthesize full-body poses based  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | on the predicted hand positions. By employing the hand positions as an          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | intermediate representation between the two denoising processes, we can         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | explicitly enforce contact constraints, resulting in more physically plausible  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | manipulation motions. With the learned model, we develop a novel system that    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | captures full-body human manipulation motions by simply attaching a smartphone  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to the object being manipulated. Through extensive experiments, we demonstrate  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the effectiveness of our proposed pipeline and its ability to generalize to     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | unseen objects. Additionally, as high-quality human-object interaction datasets |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | are scarce, we collect a large-scale dataset consisting of 3D object geometry,  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | object motion, and human motion. Our dataset contains human-object interaction  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | motion for 15 objects, with a total duration of approximately 10 hours.         |                                   |                           |
| Basis transformation properties of anomalous dimensions for hard exclusive processes                                                         | S. Van Thurenhout                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | When considering the renormalization of composite operators for the             | http://arxiv.org/abs/2309.16236v1 | 2023-09-28 08:20:55+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | description of hard exclusive scattering processes, two types of operator basis |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | called the derivative basis and the Gegenbauer basis are often used in the      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | literature. In this work we set up the explicit similarity transformations      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | between these two bases, both for quark and gluon operators. This way, one can  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | use the properties of both bases to their advantage in the computation of the   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | operator anomalous dimensions, which describe the scale dependence of           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | non-perturbative non-forward parton distributions. We provide several           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications of our framework. As an application of the gluon transformation    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | formula, we compute the one-loop non-forward purely gluonic anomalous dimension |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | matrix. For the rest of the applications we focus on the transformation formula |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of the quark operator. We derive the Gegenbauer anomalous dimensions, in the    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | limit of a large number of quark flavors $n_f$, to all orders in the strong     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | coupling $\alpha_s$, extending the computation previously performed in the      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | derivative basis. Next a numeric calculation of the two-loop anomalous          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | dimensions in the derivative basis beyond the leading-color limit is presented. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Finally, we discuss a novel way of checking existing computations of the        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | conformal anomaly based on the leading-color anomalous dimensions in the        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | derivative basis.                                                               |                                   |                           |
| Influence of Dopamine Methacrylamide on swelling behaviour and nanomechanical properties of PNIPAM microgels                                 | Sandra Forg, Xuhong Guo, Regine von Klitzing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | The combination of the catechol-containing co-monomer dopamine methacrylamide   | http://arxiv.org/abs/2309.16229v1 | 2023-09-28 08:09:52+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (DMA) with stimuli-responsive microgels such as poly(N-isopropylacrylamide)     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (PNIPAM) bears a huge potential in research and for applications due to the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | versatile properties of catechols. This research gives first detailed insights  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | into the influence of DMA on the swelling of PNIPAM microgels and the           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | correlation with their nanomechanical properties. Dynamic light scattering      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (DLS) was used to analyse the swelling behaviour of microgels in bulk solution. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | The incorporation of DMA decreases the volume phase transition temperature      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (VPTT) and completion temperature (VPT CT) due to its higher hydrophobicity     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | when compared to NIPAM, while sharpening the transition. The cross-linking      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ability of DMA decreases swelling ratios and mesh sizes of the microgels.       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Microgels adsorbed at the solid surface are characterised by atomic force       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | microscopy (AFM): Scanning provides information about the microgel's shape on   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the surface and force spectroscopy measurements determines their nanomechanical |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | properties ($E$ modulus). As the DMA content increases, microgels protrude more |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | from the surface, correlating with an increase of $E$ modulus and a stiffening  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of the microgels - confirming the cross-linking ability of DMA. Force           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | spectroscopy measurements below and above the VPTT display a stiffening of the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | microgels with the incorporation of DMA and upon heating across it's entire     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | cross-section. The affine network factor $\beta$, derived from the Flory-Rehner |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | theory describing the elasticity and swelling of the microgel network, is       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | linearly correlated with the $E$ moduli of the microgels for both - pure PNIPAM |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and P(NIPAM-co-DMA) microgels. However, for large amounts of DMA, DMA appears   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to hinder the microgel shrinking, while still ensuring mechanical stiffness,    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | possibly due to catechol interactions within the microgel network.              |                                   |                           |
| Experimental evidence of random shock-wave intermittency                                                                                     | Guillaume Ricard, Eric Falcon                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | We report the experimental observation of intermittency in a regime dominated   | http://arxiv.org/abs/2309.16222v1 | 2023-09-28 07:54:40+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | by random shock waves on the surface of a fluid. We achieved such a             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | nondispersive surface-wave field using a magnetic fluid subjected to a high     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | external magnetic field. We found that the small-scale intermittency of the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | wave-amplitude fluctuations is due to shock waves, leading to much more intense |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | intermittency than previously reported in three-dimensional hydrodynamics       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | turbulence or in wave turbulence. The statistical properties of intermittency   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | are found to be in good agreement with the predictions of a Burgerslike         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | intermittency model. Such experimental evidence of random shock-wave            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | intermittency could lead to applications in various fields.                     |                                   |                           |
| Convex Estimation of Sparse-Smooth Power Spectral Densities from Mixtures of Realizations with Application to Weather Radar                  | Hiroki Kuroda, Daichi Kitahara, Eiichi Yoshikawa, Hiroshi Kikuchi, Tomoo Ushio                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | In this paper, we propose a convex optimization-based estimation of sparse      | http://arxiv.org/abs/2309.16215v1 | 2023-09-28 07:41:14+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and smooth power spectral densities (PSDs) of complex-valued random processes   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | from mixtures of realizations. While the PSDs are related to the magnitude of   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the frequency components of the realizations, it has been a major challenge to  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | exploit the smoothness of the PSDs because penalizing the difference of the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | magnitude of the frequency components results in a nonconvex optimization       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | problem that is difficult to solve. To address this challenge, we design the    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | proposed model that jointly estimates the complex-valued frequency components   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and the nonnegative PSDs, which are respectively regularized to be sparse and   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | sparse-smooth. By penalizing the difference of the nonnegative variable that    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | estimates the PSDs, the proposed model can enhance the smoothness of the PSDs   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | via convex optimization. Numerical experiments on the phased array weather      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | radar, an advanced weather radar system, demonstrate that the proposed model    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | achieves superior estimation accuracy to the existing sparse estimation models  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | combined with or without post-smoothing.                                        |                                   |                           |
| Canary: Congestion-Aware In-Network Allreduce Using Dynamic Trees                                                                            | Daniele De Sensi, Edgar Costa Molero, Salvatore Di Girolamo, Laurent Vanbever, Torsten Hoefler                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | The allreduce operation is an essential building block for many distributed     | http://arxiv.org/abs/2309.16214v1 | 2023-09-28 07:39:25+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | applications, ranging from the training of deep learning models to scientific   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | computing. In an allreduce operation, data from multiple hosts is aggregated    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | together and then broadcasted to each host participating in the operation.      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Allreduce performance can be improved by a factor of two by aggregating the     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | data directly in the network. Switches aggregate data coming from multiple      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ports before forwarding the partially aggregated result to the next hop. In all |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | existing solutions, each switch needs to know the ports from which it will      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | receive the data to aggregate. However, this forces packets to traverse a       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | predefined set of switches, making these solutions prone to congestion. For     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | this reason, we design Canary, the first congestion-aware in-network allreduce  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | algorithm. Canary uses load balancing algorithms to forward packets on the      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | least congested paths. Because switches do not know from which ports they will  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | receive the data to aggregate, they use timeouts to aggregate the data in a     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | best-effort way. We develop a P4 Canary prototype and evaluate it on a Tofino   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | switch. We then validate Canary through simulations on large networks, showing  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | performance improvements up to 40% compared to the state-of-the-art.            |                                   |                           |
| Abdominal multi-organ segmentation in CT using Swinunter                                                                                     | Mingjin Chen, Yongkang He, Yongyi Lu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | Abdominal multi-organ segmentation in computed tomography (CT) is crucial for   | http://arxiv.org/abs/2309.16210v1 | 2023-09-28 07:32:22+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | many clinical applications including disease detection and treatment planning.  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Deep learning methods have shown unprecedented performance in this perspective. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | However, it is still quite challenging to accurately segment different organs   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | utilizing a single network due to the vague boundaries of organs, the complex   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | background, and the substantially different organ size scales. In this work we  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | used make transformer-based model for training. It was found through previous   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | years' competitions that basically all of the top 5 methods used CNN-based      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | methods, which is likely due to the lack of data volume that prevents           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | transformer-based methods from taking full advantage. The thousands of samples  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in this competition may enable the transformer-based model to have more         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | excellent results. The results on the public validation set also show that the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | transformer-based model can achieve an acceptable result and inference time.    |                                   |                           |
| Max-Sliced Mutual Information                                                                                                                | Dor Tsur, Ziv Goldfeld, Kristjan Greenewald                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | Quantifying the dependence between high-dimensional random variables is         | http://arxiv.org/abs/2309.16200v1 | 2023-09-28 06:49:25+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | central to statistical learning and inference. Two classical methods are        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | canonical correlation analysis (CCA), which identifies maximally correlated     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | projected versions of the original variables, and Shannon's mutual information, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | which is a universal dependence measure that also captures high-order           |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | dependencies. However, CCA only accounts for linear dependence, which may be    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | insufficient for certain applications, while mutual information is often        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | infeasible to compute/estimate in high dimensions. This work proposes a middle  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ground in the form of a scalable information-theoretic generalization of CCA,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | termed max-sliced mutual information (mSMI). mSMI equals the maximal mutual     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | information between low-dimensional projections of the high-dimensional         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | variables, which reduces back to CCA in the Gaussian case. It enjoys the best   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | of both worlds: capturing intricate dependencies in the data while being        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | amenable to fast computation and scalable estimation from samples. We show that |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | mSMI retains favorable structural properties of Shannon's mutual information,   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | like variational forms and identification of independence. We then study        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | statistical estimation of mSMI, propose an efficiently computable neural        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | estimator, and couple it with formal non-asymptotic error bounds. We present    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | experiments that demonstrate the utility of mSMI for several tasks,             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | encompassing independence testing, multi-view representation learning,          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | algorithmic fairness, and generative modeling. We observe that mSMI             |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | consistently outperforms competing methods with little-to-no computational      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | overhead.                                                                       |                                   |                           |
| Generation of Spatiotemporal Vortex Pulses by Simple Diffractive Grating                                                                     | Zhiyuan Che, Wenzhe Liu, Lei Shi, C. T. Chan, Jian Zi                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Spatiotemporal vortex pulses are wave packets that carry transverse orbital     | http://arxiv.org/abs/2309.16185v1 | 2023-09-28 06:03:53+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | angular momentum, exhibiting exotic structured wavefronts that can twist        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | through space and time. Existing methods to generate these pulses require       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | complex setups like spatial light modulators or computer-optimized structures.  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Here, we demonstrate a new approach to generate spatiotemporal vortex pulses    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | using just a simple diffractive grating. The key is constructing a phase vortex |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | in frequency-momentum space by leveraging symmetry, resonance, and diffraction. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Our approach is applicable to any wave system. We use a liquid surface wave     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | platform to directly demonstrate and observe the real-time generation and       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | evolution of spatiotemporal vortex pulses. This straightforward technique       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | provides opportunities to explore pulse dynamics and potential applications     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | across different disciplines.                                                   |                                   |                           |
| Large Language Model Soft Ideologization via AI-Self-Consciousness                                                                           | Xiaotian Zhou, Qian Wang, Xiaofeng Wang, Haixu Tang, Xiaozhong Liu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Large language models (LLMs) have demonstrated human-level performance on a     | http://arxiv.org/abs/2309.16167v1 | 2023-09-28 04:47:58+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | vast spectrum of natural language tasks. However, few studies have addressed    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the LLM threat and vulnerability from an ideology perspective, especially when  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | they are increasingly being deployed in sensitive domains, e.g., elections and  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | education. In this study, we explore the implications of GPT soft               |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ideologization through the use of AI-self-consciousness. By utilizing GPT       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | self-conversations, AI can be granted a vision to "comprehend" the intended     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | ideology, and subsequently generate finetuning data for LLM ideology injection. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | When compared to traditional government ideology manipulation techniques, such  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | as information censorship, LLM ideologization proves advantageous; it is easy   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to implement, cost-effective, and powerful, thus brimming with risks.           |                                   |                           |
| Doppler Time-of-Flight Rendering                                                                                                             | Juhyeon Kim, Wojciech Jarosz, Ioannis Gkioulekas, Adithya Pediredla                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | We introduce Doppler time-of-flight (D-ToF) rendering, an extension of ToF      | http://arxiv.org/abs/2309.16163v1 | 2023-09-28 04:30:51+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | rendering for dynamic scenes, with applications in simulating D-ToF cameras.    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | D-ToF cameras use high-frequency modulation of illumination and exposure, and   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | measure the Doppler frequency shift to compute the radial velocity of dynamic   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | objects. The time-varying scene geometry and high-frequency modulation          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | functions used in such cameras make it challenging to accurately and            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | efficiently simulate their measurements with existing ToF rendering algorithms. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | We overcome these challenges in a twofold manner: To achieve accuracy, we       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | derive path integral expressions for D-ToF measurements under global            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | illumination and form unbiased Monte Carlo estimates of these integrals. To     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | achieve efficiency, we develop a tailored time-path sampling technique that     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | combines antithetic time sampling with correlated path sampling. We show        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | experimentally that our sampling technique achieves up to two orders of         |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | magnitude lower variance compared to naive time-path sampling. We provide an    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | open-source simulator that serves as a digital twin for D-ToF imaging systems,  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | allowing imaging researchers, for the first time, to investigate the impact of  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | modulation functions, material properties, and global illumination on D-ToF     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | imaging performance.                                                            |                                   |                           |
| Respondent-Driven Sampling: An Overview in the Context of Human Trafficking                                                                  | Jessica P. Kunke, Adam Visokay, Tyler H. McCormick                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Respondent-driven sampling (RDS) is both a sampling strategy and an             | http://arxiv.org/abs/2309.16160v1 | 2023-09-28 04:23:14+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | estimation method. It is commonly used to study individuals that are difficult  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to access with standard sampling techniques. As with any sampling strategy, RDS |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | has advantages and challenges. This article examines recent work using RDS in   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the context of human trafficking. We begin with an overview of the RDS process  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and methodology, then discuss RDS in the particular context of trafficking. We  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | end with a description of recent work and potential future directions.          |                                   |                           |
| Magnetic Field Structure of the Crab Pulsar Wind Nebula Revealed with IXPE                                                                   | T. Mizuno, H. Ohno, E. Watanabe, N. Bucciantini, S. Gunji, S. Shibata, P. Slane, W. C. Weisskopf                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | We report a detailed study of the magnetic-field structure of the Crab pulsar   | http://arxiv.org/abs/2309.16154v1 | 2023-09-28 04:04:35+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | wind nebula, using the X-ray polarization data in 2--8~keV obtained with the    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Imaging X-ray Polarimetry Explorer. Contamination of the pulsar emission to the |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | data of the nebula region was removed through application of a stringent        |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pulsation phase-cut, extracting a phase range of 0.7--1.0 only. We found that   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the electric field vector polarization angle (PA) was about $130^{\circ}$ from  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | north to east with the polarization degree (PD) of about 25\% at the pulsar     |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | position, indicating that the direction of the toroidal magnetic field is       |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | perpendicular to the pulsar spin axis in the region close to the termination    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | shock. The PA gradually deviated from the angle as an increasing function of    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | the distance from the pulsar. There was a region of a low PD to the west of the |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | X-ray torus. Although such a region is expected to be located at the torus      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | edge, where geometrical depolarization due to a steep spatial variation of the  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | PA is expected, the observed low-PD region positionally deviated from the edge. |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | We found that the region of low PD positionally coincided with a dense filament |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | seen in the optical band, and conjecture that the low-PD region may be produced |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | through deflection of the pulsar wind. By comparing the values of the PD at the |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pulsar position between the data and a model, in which toroidal and turbulent   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | magnetic fields were considered, we estimated the fractional energy of the      |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | turbulent magnetic field to be about $2/3$ of the total. We also evaluated a    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | potential polarization of the northern jet in the nebula and derived the PD and |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | PA to be about $30\%$ and $120^{\circ}$, respectively.                          |                                   |                           |
| AE-GPT: Using Large Language Models to Extract Adverse Events from Surveillance Reports-A Use Case with Influenza Vaccine Adverse Events     | Yiming Li, Jianfu Li, Jianping He, Cui Tao                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Though Vaccines are instrumental in global health, mitigating infectious        | http://arxiv.org/abs/2309.16150v1 | 2023-09-28 03:53:21+00:00 |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | diseases and pandemic outbreaks, they can occasionally lead to adverse events   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | (AEs). Recently, Large Language Models (LLMs) have shown promise in effectively |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | identifying and cataloging AEs within clinical reports. Utilizing data from the |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | Vaccine Adverse Event Reporting System (VAERS) from 1990 to 2016, this study    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | particularly focuses on AEs to evaluate LLMs' capability for AE extraction. A   |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | variety of prevalent LLMs, including GPT-2, GPT-3 variants, GPT-4, and Llama 2, |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | were evaluated using Influenza vaccine as a use case. The fine-tuned GPT 3.5    |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | model (AE-GPT) stood out with a 0.704 averaged micro F1 score for strict match  |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | and 0.816 for relaxed match. The encouraging performance of the AE-GPT          |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | underscores LLMs' potential in processing medical data, indicating a            |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | significant stride towards advanced AE detection, thus presumably generalizable |                                   |                           |
|                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | to other AE extraction tasks.                                                   |                                   |                           |